In [ ]:
#V3 - refmaking fixed!

In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np

from sklearn.preprocessing import MinMaxScaler

In [ ]:
plt.rcParams['figure.dpi'] = 200
plt.rcParams['figure.figsize'] = (4, 4)

# Set options to display a specific number of columns and control column width
pd.set_option('display.max_columns', 20)  # Set the maximum number of columns to display
pd.set_option('display.max_colwidth', 100)  # Set the maximum column width for text data
pd.set_option('display.max_rows', 100)

In [ ]:
import os
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Define your cell types and paths
cell_types = ["Podo", "TAL", "DCT_CNT_CD", "EC", "Stromal", "Immune", "PEC", "PT", "IC", "DTL_ATL"]  # etc.
base_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/human_extended/output_human_extended/"
r2_folder = "R2_GO_unfiltered/"
pval_folder = "Padj_GO_unfiltered/"
metadat_path = "/home/isilon/users/o_kloetzer/Atlas/scSPECTRA/R2_pval/Atlas_Extended_II_Albuminuria_gpt.csv"  # Update with the actual path

def extract_cell_type(gene_set):
    for cell_type in cell_types:
        if gene_set.startswith(cell_type):
            return cell_type
    return "Unknown"  # or some default value

# Read metadata
metadat = pd.read_csv(metadat_path)
unique_projects = metadat['Project/Dataset'].unique()
#unique_projects = ["h_KPMP", "cal_CKD"]

# Initialize a DataFrame to store the average normalized percentage for each feature
average_normalized_percentage_df = pd.DataFrame()

for project in unique_projects:
    unique_samples = metadat[(metadat['Disease_level2'] != "Control") & (metadat['Project/Dataset'] == project)]['Sample'].unique()

    # Initialize combined dataframes for R2 and p-values for this project
    combined_r2 = pd.DataFrame(index=unique_samples)
    combined_pval = pd.DataFrame(index=unique_samples)
    sample_counts_per_cell_type = {}

    for cell_type in cell_types:
        r2_path = os.path.join(base_path, r2_folder, f"R2_{cell_type}.csv")
        pval_path = os.path.join(base_path, pval_folder, f"Pval_{cell_type}.csv")
        
        r2_df = pd.read_csv(r2_path, index_col=0)
        pval_df = pd.read_csv(pval_path, index_col=0)

        # Modify column names
        r2_df.columns = [f"{cell_type}_{col}" for col in r2_df.columns]
        pval_df.columns = [f"{cell_type}_{col}" for col in pval_df.columns]
        
        # Filter r2_df to only include rows where the index is in unique_samples
        r2_df = r2_df[r2_df.index.isin(unique_samples)]

        # Merge data
        combined_r2 = combined_r2.join(r2_df, how='left')
        combined_pval = combined_pval.join(pval_df, how='left')

        # Count samples per cell type
        sample_counts_per_cell_type[cell_type] = r2_df.shape[0]

    # Handle NA values in p-values
    combined_pval.fillna(1, inplace=True)

    # Initialize a MinMaxScaler and scale the combined_r2 data
    min_max_scaler = MinMaxScaler()
    min_max_scaled_r2_values = min_max_scaler.fit_transform(combined_r2.values)
    combined_r2 = pd.DataFrame(min_max_scaled_r2_values, index=combined_r2.index, columns=combined_r2.columns)

    # Set p-value threshold and identify significant gene sets
    p_value_threshold = 0.05
    significant_matrix = combined_pval < p_value_threshold
    significant_counts = significant_matrix.sum()

    # Create DataFrame for counts and cell types
    significant_counts_df = pd.DataFrame({'GeneSet': significant_counts.index, 'Count': significant_counts.values})
    significant_counts_df['CellType'] = significant_counts_df['GeneSet'].apply(extract_cell_type)

    # Add a column for the normalized percentage for the current project
    project_column = f"NormalizedPercentage_{project}"
    significant_counts_df[project_column] = significant_counts_df.apply(
        lambda row: (row['Count'] / sample_counts_per_cell_type.get(row['CellType'], 1)) * 100, axis=1
    )

    if average_normalized_percentage_df.empty:
        # For the first project, initialize the DataFrame
        average_normalized_percentage_df = significant_counts_df[['GeneSet', project_column]]
    else:
        # For subsequent projects, merge their data into the DataFrame
        average_normalized_percentage_df = average_normalized_percentage_df.merge(
            significant_counts_df[['GeneSet', project_column]],
            on='GeneSet',
            how='outer'
        )

# After all projects are processed, calculate the average
average_column = 'AverageNormalizedPercentage'
average_normalized_percentage_df[average_column] = average_normalized_percentage_df.filter(like='NormalizedPercentage_').mean(axis=1)

# Sort the features based on the average normalized percentage
sorted_average_df = average_normalized_percentage_df.sort_values(by=average_column, ascending=False)

# Display the sorted DataFrame
print(sorted_average_df.head())  # Modify this to display as needed



In [ ]:
sorted_average_df[:100]

In [ ]:

# Replace with your actual file paths and cell types
cell_types = ["Podo", "TAL", "DCT_CNT_CD", "EC", "Stromal", "Immune", "PEC", "PT", "IC", "DTL_ATL"] # etc.
# Paths as specified in your first script
base_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/human_extended/output_human_extended/"
r2_folder = "R2_GO_unfiltered/"
pval_folder = "Padj_GO_unfiltered/"
metadat_path = "/home/isilon/users/o_kloetzer/Atlas/scSPECTRA/R2_pval/Atlas_Extended_II_Albuminuria_gpt.csv"  # Update with the actual path to 'metadat'

def extract_cell_type(gene_set):
    for cell_type in cell_types:
        if gene_set.startswith(cell_type):
            return cell_type
    return "Unknown"  # or some default value

# Read metadata
metadat = pd.read_csv(metadat_path)
unique_samples = metadat[metadat['Disease_level2'] != "Control"]['Sample'].unique()
#unique_samples = metadat[(metadat['Disease_level2'] != "Control") & (metadat['Project/Dataset'] == "cal_CKD")]['Sample'].unique()

# Initialize combined dataframes for R2 and p-values
combined_r2 = pd.DataFrame(index=unique_samples)
combined_pval = pd.DataFrame(index=unique_samples)
sample_counts_per_cell_type = {}

# Process each cell type
for cell_type in cell_types:
    r2_path = os.path.join(base_path, r2_folder, f"R2_{cell_type}.csv")
    pval_path = os.path.join(base_path, pval_folder, f"Pval_{cell_type}.csv")
    
    r2_df = pd.read_csv(r2_path, index_col=0)
    pval_df = pd.read_csv(pval_path, index_col=0)
    
    # Modify column names
    r2_df.columns = [f"{cell_type}_{col}" for col in r2_df.columns]
    pval_df.columns = [f"{cell_type}_{col}" for col in pval_df.columns]
    
    # Merge data
    combined_r2 = combined_r2.join(r2_df, how='left')
    combined_pval = combined_pval.join(pval_df, how='left')

    # Count samples per cell type
    r2_df = r2_df[r2_df.index.isin(unique_samples)]
    sample_counts_per_cell_type[cell_type] = r2_df.shape[0]

# Handle NA values in p-values
combined_pval.fillna(1, inplace=True)

print(combined_r2)

# Initialize a MinMaxScaler
min_max_scaler = MinMaxScaler()

# Scale the combined_r2 data
# Note: The fit_transform method expects a 2D array, so we use .values to convert the DataFrame
min_max_scaled_r2_values = min_max_scaler.fit_transform(combined_r2.values)

# Create a new DataFrame from the min-max scaled data, preserving the index and column names

combined_r2 = pd.DataFrame(min_max_scaled_r2_values, index=combined_r2.index, columns=combined_r2.columns)

# Set p-value threshold
p_value_threshold = 0.05

# Identify significant gene sets
significant_matrix = combined_pval < p_value_threshold

# Count significant occurrences
significant_counts = significant_matrix.sum()

# Create DataFrame for counts and cell types
significant_counts_df = pd.DataFrame({'GeneSet': significant_counts.index, 'Count': significant_counts.values})

# Apply the function to extract the full cell type name
significant_counts_df['CellType'] = significant_counts_df['GeneSet'].apply(extract_cell_type)

# Normalize by sample number per cell type
significant_counts_df['NormalizedPercentage'] = significant_counts_df.apply(
    lambda row: (row['Count'] / sample_counts_per_cell_type.get(row['CellType'], 1)) * 100, axis=1
)

# Display final DataFrame
print(significant_counts_df.head())  # You can modify this to display as needed

sorted_df = significant_counts_df.sort_values(by='NormalizedPercentage', ascending=False)
display(sorted_df)

In [ ]:


sample_of_interest = 'KRAD60'

#max_value = 300

ct_plot1 = "Podo"

ct_plot2 = "PEC"

ct_plot3 = "PT"


# Replace with your actual file paths and cell types
cell_types = ["Podo", "TAL", "DCT_CNT_CD", "EC", "Stromal", "Immune", "PEC", "PT", "IC", "DTL_ATL"] # etc.
# Paths as specified in your first script
base_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/human_extended/output_human_extended/"
r2_folder = "R2_GO_unfiltered/"
pval_folder = "Padj_GO_unfiltered/"
metadat_path = "/home/isilon/users/o_kloetzer/Atlas/scSPECTRA/R2_pval/Atlas_Extended_II_Albuminuria_gpt.csv"  # Update with the actual path to 'metadat'

def extract_cell_type(gene_set):
    for cell_type in cell_types:
        if gene_set.startswith(cell_type):
            return cell_type
    return "Unknown"  # or some default value

# Read metadata
metadat = pd.read_csv(metadat_path)
unique_samples = metadat[metadat['Disease_level2'] != "Control"]['Sample'].unique()
#unique_samples = metadat[(metadat['Disease_level2'] != "Control") & (metadat['Project/Dataset'] == "cal_CKD")]['Sample'].unique()

# Initialize combined dataframes for R2 and p-values
combined_r2 = pd.DataFrame(index=unique_samples)
combined_pval = pd.DataFrame(index=unique_samples)
sample_counts_per_cell_type = {}

# Process each cell type
for cell_type in cell_types:
    r2_path = os.path.join(base_path, r2_folder, f"R2_{cell_type}.csv")
    pval_path = os.path.join(base_path, pval_folder, f"Pval_{cell_type}.csv")
    
    r2_df = pd.read_csv(r2_path, index_col=0)
    pval_df = pd.read_csv(pval_path, index_col=0)
    
    # Modify column names
    r2_df.columns = [f"{cell_type}_{col}" for col in r2_df.columns]
    pval_df.columns = [f"{cell_type}_{col}" for col in pval_df.columns]
    
    # Merge data
    combined_r2 = combined_r2.join(r2_df, how='left')
    combined_pval = combined_pval.join(pval_df, how='left')

    # Count samples per cell type
    r2_df = r2_df[r2_df.index.isin(unique_samples)]
    sample_counts_per_cell_type[cell_type] = r2_df.shape[0]

# Handle NA values in p-values
combined_pval.fillna(1, inplace=True)

print(combined_r2)

# Initialize a MinMaxScaler
min_max_scaler = MinMaxScaler()

# Scale the combined_r2 data
# Note: The fit_transform method expects a 2D array, so we use .values to convert the DataFrame
min_max_scaled_r2_values = min_max_scaler.fit_transform(combined_r2.values)

# Create a new DataFrame from the min-max scaled data, preserving the index and column names

combined_r2 = pd.DataFrame(min_max_scaled_r2_values, index=combined_r2.index, columns=combined_r2.columns)

# Set p-value threshold
p_value_threshold = 0.05

# Identify significant gene sets
significant_matrix = combined_pval < p_value_threshold

# Count significant occurrences
significant_counts = significant_matrix.sum()

# Create DataFrame for counts and cell types
significant_counts_df = pd.DataFrame({'GeneSet': significant_counts.index, 'Count': significant_counts.values})

# Apply the function to extract the full cell type name
significant_counts_df['CellType'] = significant_counts_df['GeneSet'].apply(extract_cell_type)

# Normalize by sample number per cell type
significant_counts_df['NormalizedPercentage'] = significant_counts_df.apply(
    lambda row: (row['Count'] / sample_counts_per_cell_type.get(row['CellType'], 1)) * 100, axis=1
)

# Display final DataFrame
print(significant_counts_df.head())  # You can modify this to display as needed

sorted_df = significant_counts_df.sort_values(by='NormalizedPercentage', ascending=False)
display(sorted_df)

import re

def auto_split_label(label, max_length=40):
    # Split the label into words
    words = label.split()

    # Function to remove suffix
    def remove_suffix(word):
        return re.sub(r'\(GO:\d{7}\)$', '', word)

    # Start with the first word (with suffix removed)
    first_word = remove_suffix(words[0])
    split_label = first_word
    current_length = len(first_word)

    # Add each word to the line until the max_length is reached, then start a new line
    for word in words[1:]:
        word = remove_suffix(word)
        if current_length + len(word) + 1 <= max_length:
            split_label += ' ' + word
            current_length += len(word) + 1
        else:
            split_label += '\n' + word
            current_length = len(word)

    return split_label


# Provided cell types and colors
cell_colors = {
    "DCT_CNT_CD": "#3182bd",
    "DTL_ATL": "#fdd0a2",
    "EC": "seagreen",
    "IC": "orange",
    "Immune": "#c7e9c0",
    "Podo": "#000004",
    "Stromal": "limegreen",
    "PEC": "#fde725",
    "PT": "darkorchid",
    "TAL": "lightcoral",
}

# Function to extract cell type from feature name
def extract_cell_type(feature_name):
    for cell_type in cell_colors.keys():
        if feature_name.startswith(cell_type):
            return cell_type
    return "Unknown"

# Sample data - replace with your actual data

top_20_features = combined_pval.loc[sample_of_interest].nsmallest(25)
neg_log_pvals = -np.log10(top_20_features)
r2_values = combined_r2.loc[sample_of_interest][top_20_features.index]

# Create a reversed red colormap for R2 values
cmap = plt.cm.Reds_r  # '_r' suffix to reverse the colormap

norm = mcolors.Normalize(vmin=0, vmax=1)  # Normalization fixed from 0 to 1

# Assuming 'significant_counts_df' is your DataFrame and it has a 'GeneSet' and 'NormalizedPercentage' column
scale_factor = 4
base_size = 100
#base_size = 20

# Map features to their normalized percentage
feature_to_percentage = dict(zip(significant_counts_df['GeneSet'], significant_counts_df['NormalizedPercentage']))

# Create the dot plot
fig, ax = plt.subplots(figsize=(16, 5))
for i, (feature, neg_log_pval) in enumerate(neg_log_pvals.items()):
    cell_type = extract_cell_type(feature)
    bar_color = cell_colors.get(cell_type, 'grey')
    dot_color = cmap(norm(r2_values[feature]))
    
    # Get the normalized percentage for the dot size
    normalized_percentage = feature_to_percentage.get(feature, 0)
    dot_size = normalized_percentage * scale_factor + base_size

    ax.plot([0, neg_log_pval - (neg_log_pval / 10)], [i, i], color=bar_color, linewidth=3)
    ax.scatter(neg_log_pval, i, s=dot_size, color=dot_color, edgecolor='black', alpha=0.7, linewidth=0.5)

ax.set_yticks(range(len(top_20_features.index)))
ax.set_yticklabels(top_20_features.index, fontsize=10)
ax.set_xlabel('-log10(p-value)', fontsize=12)
ax.set_ylabel('Features', fontsize=12)
ax.set_title(f'Top Features for Sample {sample_of_interest}', fontsize=14)

# Invert y-axis to have the most significant features at the top
ax.invert_yaxis()

# Create a colorbar for the R2 values
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax, label='Scaled R2 Values', orientation='vertical')

# Legend for cell types
cell_legend_elements = [plt.Line2D([0], [0], color=color, lw=4, label=cell_type) for cell_type, color in cell_colors.items()]
legend1 = ax.legend(handles=cell_legend_elements, title='Cell Types', bbox_to_anchor=(1.5, 1), loc='upper left')

# Example normalized percentages for the dot size legend
example_percentages = [20, 40, 60, 80]
legend_dot_sizes = [p * scale_factor + base_size for p in example_percentages]

# Add legend for dot sizes
for p, size in zip(example_percentages, legend_dot_sizes):
    ax.scatter([], [], s=size, color='gray', edgecolor='black', alpha=0.7, label=f'{p}%')

legend2 = ax.legend(title='Frequency', bbox_to_anchor=(1.5, 0.2), loc='center left')
ax.add_artist(legend1)  # Add back the first legend

plt.tight_layout()
plt.show()


# Define your desired order of cell types
desired_order = [
     
"PT", 
"TAL", 
"DCT_CNT_CD", 
"IC",    
"EC",
"Stromal",      
"Immune",   
"DTL_ATL",  
"PEC",           
"Podo",    

    
] 

sample_features = combined_pval.loc[sample_of_interest]

top_20_features = sample_features[sample_features < p_value_threshold]

# Assume 'combined_pval' is a DataFrame you have that contains the p-values

# Select the top 50 gene sets from the top features
top_gene_sets = top_20_features.index

# Group the top gene sets by cell type
grouped_data = pd.Series(top_gene_sets).apply(extract_cell_type).value_counts()

categories = list(grouped_data.index)

# Initialize ordered values with zero for all categories
ordered_values = [grouped_data.get(ct, 0) for ct in desired_order]

# Since we need to repeat the first value to close the circular graph
ordered_values += ordered_values[:1]

# Calculate the angle of each axis in the plot
N = len(desired_order)
angles = [n / float(N) * 2 * np.pi for n in range(N)]
angles += angles[:1]

# Initialize the spider plot
fig, ax = plt.subplots(figsize=(4, 4), subplot_kw=dict(polar=True))

# Draw one axis per variable and add labels
plt.xticks(angles[:-1], desired_order)

# Draw ylabels and set plot limits
ax.set_rlabel_position(0)
plt.yticks(color="grey", size=5)


#plt.ylim(0, max_value)


# Neutral color for shading
shade_color = 'lightgrey'

# Fill the entire area under the radar chart with a neutral color
ax.fill(angles, ordered_values, shade_color, alpha=0.9)

# Draw thin lines connecting the points
ax.plot(angles, ordered_values, color='grey', linewidth=1, linestyle='-', alpha=0.8)

# Plot each line segment in its respective cell type color
for idx in range(N):
    color = cell_colors.get(desired_order[idx], "grey")
    ax.plot([angles[idx], 0], [ordered_values[idx], 0], color=color, linewidth=2)

# Change the color of tick labels to red if the cell type is missing
for idx, label in enumerate(ax.get_xticklabels()):
    if desired_order[idx] not in categories:
        label.set_color('red')

# Add a title
plt.title('Cell Type Distribution in Top Gene Sets for Sample ' + sample_of_interest, size=11, y=1.1)

plt.show()


# Desired cell type for visualization
desired_cell_type = ct_plot1

# Assuming 'combined_pval' and 'combined_r2' are DataFrames with samples as rows and features as columns
# Filter for the specific sample of interest
sample_pvals = combined_pval.loc[sample_of_interest]
sample_r2 = combined_r2.loc[sample_of_interest]

# Filter the top features for the specific sample and cell type
#top_features_for_sample = sample_pvals.nsmallest(25)
filtered_features = sample_pvals[sample_pvals.index.map(extract_cell_type) == desired_cell_type].nsmallest(5)
filtered_neg_log_pvals = -np.log10(filtered_features)
filtered_r2_values = sample_r2[filtered_features.index]

# Now, update the dot plot code to use the filtered data
fig, ax = plt.subplots(figsize=(16, 5))
for i, (feature, neg_log_pval) in enumerate(filtered_neg_log_pvals.items()):
    cell_type = extract_cell_type(feature)
    bar_color = cell_colors.get(cell_type, 'grey')
    dot_color = cmap(norm(filtered_r2_values[feature]))
    
    # Get the normalized percentage for the dot size (if applicable)
    normalized_percentage = feature_to_percentage.get(feature, 0)
    dot_size = normalized_percentage * scale_factor + base_size

    ax.plot([0, neg_log_pval - (neg_log_pval / 10)], [i, i], color=bar_color, linewidth=3)
    ax.scatter(neg_log_pval, i, s=dot_size, color=dot_color, edgecolor='black', alpha=0.7, linewidth=0.5)

ax.set_yticks(range(len(filtered_features.index)))

prefix_to_remove = f"{desired_cell_type}_"
cleaned_labels = [label.replace(prefix_to_remove, '', 1) for label in filtered_features.index]

# Apply the auto-splitting to each label
cleaned_labels = [auto_split_label(label) for label in cleaned_labels]

ax.set_yticklabels(cleaned_labels, fontsize=22)
ax.set_xlabel('-log10(p-value)', fontsize=15)
#ax.set_ylabel('Features', fontsize=12)
#ax.set_title(f'Top Features for Sample {sample_of_interest} - Cell Type: {desired_cell_type}', fontsize=14)
ax.invert_yaxis()

# Create a colorbar for the R2 values
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax, label='Scaled R2 Values', orientation='vertical')

# Legend for cell types
#cell_legend_elements = [plt.Line2D([0], [0], color=color, lw=4, label=cell_type) for cell_type, color in cell_colors.items()]
#legend1 = ax.legend(handles=cell_legend_elements, title='Cell Types', bbox_to_anchor=(1.5, 1), loc='upper left')

# Example normalized percentages for the dot size legend
example_percentages = [20, 40, 60, 80]
legend_dot_sizes = [p * scale_factor + base_size for p in example_percentages]

# Add legend for dot sizes
for p, size in zip(example_percentages, legend_dot_sizes):
    ax.scatter([], [], s=size, color='gray', edgecolor='black', alpha=0.7, label=f'{p}%')

legend2 = ax.legend(title='Frequency', bbox_to_anchor=(1.5, 0.2), loc='center left', fontsize='large')
#ax.add_artist(legend1)  # Add back the first legend

plt.tight_layout()
plt.show()




# Desired cell type for visualization
desired_cell_type = ct_plot2

# Assuming 'combined_pval' and 'combined_r2' are DataFrames with samples as rows and features as columns
# Filter for the specific sample of interest
sample_pvals = combined_pval.loc[sample_of_interest]
sample_r2 = combined_r2.loc[sample_of_interest]

# Filter the top features for the specific sample and cell type
#top_features_for_sample = sample_pvals.nsmallest(25)
filtered_features = sample_pvals[sample_pvals.index.map(extract_cell_type) == desired_cell_type].nsmallest(5)
filtered_neg_log_pvals = -np.log10(filtered_features)
filtered_r2_values = sample_r2[filtered_features.index]

# Now, update the dot plot code to use the filtered data
fig, ax = plt.subplots(figsize=(16, 5))
for i, (feature, neg_log_pval) in enumerate(filtered_neg_log_pvals.items()):
    cell_type = extract_cell_type(feature)
    bar_color = cell_colors.get(cell_type, 'grey')
    dot_color = cmap(norm(filtered_r2_values[feature]))
    
    # Get the normalized percentage for the dot size (if applicable)
    normalized_percentage = feature_to_percentage.get(feature, 0)
    dot_size = normalized_percentage * scale_factor + base_size

    ax.plot([0, neg_log_pval - (neg_log_pval / 10)], [i, i], color=bar_color, linewidth=3)
    ax.scatter(neg_log_pval, i, s=dot_size, color=dot_color, edgecolor='black', alpha=0.7, linewidth=0.5)

ax.set_yticks(range(len(filtered_features.index)))

prefix_to_remove = f"{desired_cell_type}_"
cleaned_labels = [label.replace(prefix_to_remove, '', 1) for label in filtered_features.index]

# Apply the auto-splitting to each label
cleaned_labels = [auto_split_label(label) for label in cleaned_labels]

ax.set_yticklabels(cleaned_labels, fontsize=22)
ax.set_xlabel('-log10(p-value)', fontsize=15)
#ax.set_ylabel('Features', fontsize=12)
#ax.set_title(f'Top Features for Sample {sample_of_interest} - Cell Type: {desired_cell_type}', fontsize=14)
ax.invert_yaxis()

# Create a colorbar for the R2 values
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax, label='Scaled R2 Values', orientation='vertical')

# Legend for cell types
#cell_legend_elements = [plt.Line2D([0], [0], color=color, lw=4, label=cell_type) for cell_type, color in cell_colors.items()]
#legend1 = ax.legend(handles=cell_legend_elements, title='Cell Types', bbox_to_anchor=(1.5, 1), loc='upper left')

# Example normalized percentages for the dot size legend
example_percentages = [20, 40, 60, 80]
legend_dot_sizes = [p * scale_factor + base_size for p in example_percentages]

# Add legend for dot sizes
for p, size in zip(example_percentages, legend_dot_sizes):
    ax.scatter([], [], s=size, color='gray', edgecolor='black', alpha=0.7, label=f'{p}%')

legend2 = ax.legend(title='Frequency', bbox_to_anchor=(1.5, 0.2), loc='center left', fontsize='large')
#ax.add_artist(legend1)  # Add back the first legend

plt.tight_layout()
plt.show()





# Desired cell type for visualization
desired_cell_type = ct_plot3

# Assuming 'combined_pval' and 'combined_r2' are DataFrames with samples as rows and features as columns
# Filter for the specific sample of interest
sample_pvals = combined_pval.loc[sample_of_interest]
sample_r2 = combined_r2.loc[sample_of_interest]

# Filter the top features for the specific sample and cell type
#top_features_for_sample = sample_pvals.nsmallest(25)
filtered_features = sample_pvals[sample_pvals.index.map(extract_cell_type) == desired_cell_type].nsmallest(5)
filtered_neg_log_pvals = -np.log10(filtered_features)
filtered_r2_values = sample_r2[filtered_features.index]

# Now, update the dot plot code to use the filtered data
fig, ax = plt.subplots(figsize=(16, 5))
for i, (feature, neg_log_pval) in enumerate(filtered_neg_log_pvals.items()):
    cell_type = extract_cell_type(feature)
    bar_color = cell_colors.get(cell_type, 'grey')
    dot_color = cmap(norm(filtered_r2_values[feature]))
    
    # Get the normalized percentage for the dot size (if applicable)
    normalized_percentage = feature_to_percentage.get(feature, 0)
    dot_size = normalized_percentage * scale_factor + base_size

    ax.plot([0, neg_log_pval - (neg_log_pval / 10)], [i, i], color=bar_color, linewidth=3)
    ax.scatter(neg_log_pval, i, s=dot_size, color=dot_color, edgecolor='black', alpha=0.7, linewidth=0.5)

ax.set_yticks(range(len(filtered_features.index)))

prefix_to_remove = f"{desired_cell_type}_"
cleaned_labels = [label.replace(prefix_to_remove, '', 1) for label in filtered_features.index]

# Apply the auto-splitting to each label
cleaned_labels = [auto_split_label(label) for label in cleaned_labels]

ax.set_yticklabels(cleaned_labels, fontsize=22)
ax.set_xlabel('-log10(p-value)', fontsize=15)
#ax.set_ylabel('Features', fontsize=12)
#ax.set_title(f'Top Features for Sample {sample_of_interest} - Cell Type: {desired_cell_type}', fontsize=14)
ax.invert_yaxis()

# Create a colorbar for the R2 values
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax, label='Scaled R2 Values', orientation='vertical')

# Legend for cell types
#cell_legend_elements = [plt.Line2D([0], [0], color=color, lw=4, label=cell_type) for cell_type, color in cell_colors.items()]
#legend1 = ax.legend(handles=cell_legend_elements, title='Cell Types', bbox_to_anchor=(1.5, 1), loc='upper left')

# Example normalized percentages for the dot size legend
example_percentages = [20, 40, 60, 80]
legend_dot_sizes = [p * scale_factor + base_size for p in example_percentages]

# Add legend for dot sizes
for p, size in zip(example_percentages, legend_dot_sizes):
    ax.scatter([], [], s=size, color='gray', edgecolor='black', alpha=0.7, label=f'{p}%')

legend2 = ax.legend(title='Frequency', bbox_to_anchor=(1.5, 0.2), loc='center left', fontsize='large')
#ax.add_artist(legend1)  # Add back the first legend

plt.tight_layout()
plt.show()

In [ ]:
sorted_df[:50]

In [ ]:
sorted_df[:50]

In [ ]:
# Assuming 'sorted_df' is your DataFrame
# Select the top 100 features
repetitive_features = sorted_df[sorted_df["NormalizedPercentage"] >= 25]

# Remove the CellType prefix from the GeneSet column
repetitive_features['GeneSetOnly'] = repetitive_features.apply(
    lambda row: row['GeneSet'].replace(row['CellType'] + '_', ''), axis=1
)

repetitive_features

In [ ]:
len(repetitive_features["GeneSetOnly"].unique())

len(repetitive_features["GeneSetOnly"])

In [ ]:
repetitive_features.to_csv("/home/isilon/users/o_kloetzer/Atlas/Revision/human_extended/output_human_extended/Frequent_features_disease_R1.csv")

In [ ]:
import pandas as pd

# Assuming 'repetitive_features' is your DataFrame
# Count the frequency of each gene set across different cell types
gene_set_frequencies = repetitive_features.groupby('GeneSetOnly')['CellType'].nunique().reset_index()

# Rename the columns for clarity
gene_set_frequencies.columns = ['GeneSet', 'NumberOfCellTypes']

# Now, gene_set_frequencies DataFrame contains each gene set and the number of cell types it's observed in
# Sort the DataFrame in descending order based on the number of cell types
gene_set_frequencies_sorted = gene_set_frequencies.sort_values(by='NumberOfCellTypes', ascending=False)

# Now, gene_set_frequencies_sorted is sorted with the most frequently observed gene sets at the top
gene_set_frequencies_sorted[:50]

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

import re

def auto_split_label(label, max_length=40):
    # Split the label into words
    words = label.split()

    # Function to remove suffix
    def remove_suffix(word):
        return re.sub(r'\(GO:\d{7}\)$', '', word)

    # Start with the first word (with suffix removed)
    first_word = remove_suffix(words[0])
    split_label = first_word
    current_length = len(first_word)

    # Add each word to the line until the max_length is reached, then start a new line
    for word in words[1:]:
        word = remove_suffix(word)
        if current_length + len(word) + 1 <= max_length:
            split_label += ' ' + word
            current_length += len(word) + 1
        else:
            split_label += '\n' + word
            current_length = len(word)

    return split_label

# Assuming 'repetitive_features' is your DataFrame
# Identify the top common feature for each cell type
top_features_per_celltype = repetitive_features.loc[repetitive_features.groupby('CellType')['NormalizedPercentage'].idxmax()]

# Sort the DataFrame based on NormalizedPercentage in ascending order for horizontal plot
top_features_per_celltype_sorted = top_features_per_celltype.sort_values(by='NormalizedPercentage', ascending=True)

# Your specific color map for cell types
cell_colors = {
    "DCT_CNT_CD": "#3182bd",
    "DTL_ATL": "#fdd0a2",
    "EC": "seagreen",
    "IC": "orange",
    "Immune": "#c7e9c0",
    "Podo": "#555555",
    "Stromal": "limegreen",
    "PEC": "#fde725",
    "PT": "darkorchid",
    "TAL": "lightcoral",
}

# Plotting the horizontal bar plot
plt.figure(figsize=(12, 8))
barplot = plt.barh(top_features_per_celltype_sorted['CellType'], top_features_per_celltype_sorted['NormalizedPercentage'],
                   color=[cell_colors[ct] for ct in top_features_per_celltype_sorted['CellType']])

# Setting the y-axis labels using the auto_split_label function
plt.yticks(top_features_per_celltype_sorted['CellType'], 
           [auto_split_label(gs) for gs in top_features_per_celltype_sorted['GeneSetOnly']], fontsize = 18)

plt.xlabel('Normalized Percentage')
plt.title('Top Common Features per Cell Type (Sorted by Normalized Percentage)')
plt.tight_layout()  # Adjusts the plot to ensure everything fits without overlapping
plt.show()



In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Function to split long labels
def auto_split_label(label, max_length=50):
    words = label.split()
    split_label = words[0]
    current_length = len(words[0])

    for word in words[1:]:
        if current_length + len(word) + 1 <= max_length:
            split_label += ' ' + word
            current_length += len(word) + 1
        else:
            split_label += '\n' + word
            current_length = len(word)

    return split_label

# Assuming 'sorted_df' is your DataFrame with columns ['CellType', 'GeneSet', 'NormalizedPercentage']

# Create a new column 'GeneSetOnly' by removing the cell type prefix from 'GeneSet'
sorted_df['GeneSetOnly'] = sorted_df.apply(lambda row: row['GeneSet'].replace(row['CellType'] + '_', ''), axis=1)

# Filter for top 100 features
top_100_features = sorted_df[sorted_df["NormalizedPercentage"] >= 25]

# Identify cell-type specific gene sets
unique_gene_sets = top_100_features.groupby('GeneSetOnly').filter(lambda x: len(x['CellType'].unique()) == 1)

# Identify the top common feature for each cell type
top_features_per_celltype = unique_gene_sets.loc[unique_gene_sets.groupby('CellType')['NormalizedPercentage'].idxmax()]

# Sort the DataFrame based on NormalizedPercentage in ascending order for horizontal plot
top_features_per_celltype_sorted = top_features_per_celltype.sort_values(by='NormalizedPercentage', ascending=True)

# Your specific color map for cell types
cell_colors = {
    "DCT_CNT_CD": "#3182bd",
    "DTL_ATL": "#fdd0a2",
    "EC": "seagreen",
    "IC": "orange",
    "Immune": "#c7e9c0",
    "Podo": "#555555",
    "Stromal": "limegreen",
    "PEC": "#fde725",
    "PT": "darkorchid",
    "TAL": "lightcoral",
}

# Plotting the horizontal bar plot
plt.figure(figsize=(12, 8))
plt.barh(top_features_per_celltype_sorted['CellType'], top_features_per_celltype_sorted['NormalizedPercentage'],
         color=[cell_colors[ct] for ct in top_features_per_celltype_sorted['CellType']])

# Setting the y-axis labels using the auto_split_label function
plt.yticks(top_features_per_celltype_sorted['CellType'], 
           [auto_split_label(gs) for gs in top_features_per_celltype_sorted['GeneSetOnly']], fontsize=18)

plt.xlabel('Normalized Percentage')
plt.title('Top Cell-Type Specific Features per Cell Type (Sorted by Normalized Percentage)')
plt.tight_layout()
plt.show()



In [ ]:
cell_colors = {
    "DCT_CNT_CD": "#3182bd",
    "DTL_ATL": "#fdd0a2",
    "EC": "seagreen",
    "IC": "orange",
    "Immune": "#c7e9c0",
    "Podo": "#555555",
    "Stromal": "limegreen",
    "PEC": "#fde725",
    "PT": "darkorchid",
    "TAL": "lightcoral",
}

# Assuming 'sorted_conserved_features' is already available from previous steps

# Select the top 100 features
top_100_features = sorted_df[sorted_df["NormalizedPercentage"] >= 25]

# Count the frequency of each CellType in the top 100 features
cell_type_counts = top_100_features['CellType'].value_counts()

# Extract the colors corresponding to the cell types
pie_colors = [cell_colors[cell_type] for cell_type in cell_type_counts.index if cell_type in cell_colors]

# Plotting the pie chart
plt.figure(figsize=(5, 5))  # Adjust the size as needed
cell_type_counts.plot(kind='pie', colors=pie_colors, autopct='')
plt.title('25% of patients cut-off')
plt.ylabel(len(top_100_features))  # Hide the y-label
plt.show()

In [ ]:
metadata

In [ ]:
p_value_threshold = 0.05

sample_of_interest = "KRAD70"

#max_value = 300

ct_plot1 = "PT"

#specify how many nan we tolerate (critical if applying gene filtering during spectra analysis)
nan_threshold = 1000

import pandas as pd
import os

# Define cell types and file paths
cell_types = ["PT", "Podo", "TAL", "DCT_CNT_CD", "EC", "Stromal", "IC", "Immune", "PEC", "DTL_ATL"]

#cell_types = ["PEC", "Immune"]

base_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/human_extended/output_human_extended/"
r2_folder = "R2_GO_unfiltered/"
pval_folder = "Padj_GO_unfiltered/"
metadat_path = "/home/isilon/users/o_kloetzer/Atlas/scSPECTRA/R2_pval/Atlas_Extended_II_Albuminuria_gpt.csv"  # Update with the actual path to 'metadat'
metadata_path = metadat_path

def extract_cell_type(gene_set):
    for cell_type in cell_types:
        if gene_set.startswith(cell_type):
            return cell_type
    return "Unknown"

# Read metadata
metadata = pd.read_csv(metadata_path)
unique_samples = metadata[metadata['Disease_level1'] != "Control"]['Sample'].unique()
#sample_to_species = metadata.set_index('orig_ident')['species'].to_dict()

# Initialize combined dataframes for R2 and p-values
combined_r2 = pd.DataFrame(index=unique_samples)
combined_pval = pd.DataFrame(index=unique_samples)
sample_counts_per_cell_type = {}
#sample_counts_per_cell_type_per_species = {species: {ct: 0 for ct in cell_types} for species in metadata['species'].unique()}

# Process each cell type
for cell_type in cell_types:
    r2_path = os.path.join(base_path, r2_folder, f"R2_{cell_type}.csv")
    pval_path = os.path.join(base_path, pval_folder, f"Pval_{cell_type}.csv")
    
    r2_df = pd.read_csv(r2_path, index_col=0)
    r2_df = r2_df[r2_df.index.isin(unique_samples)]
    pval_df = pd.read_csv(pval_path, index_col=0)
    pval_df = pval_df[pval_df.index.isin(unique_samples)]
    
    r2_df.columns = [f"{cell_type}_{col}" for col in r2_df.columns]
    pval_df.columns = [f"{cell_type}_{col}" for col in pval_df.columns]
    
    combined_r2 = combined_r2.join(r2_df, how='left')
    combined_pval = combined_pval.join(pval_df, how='left')

    sample_counts_per_cell_type[cell_type] = r2_df.shape[0]
    #for sample in r2_df.index:
        #species = sample_to_species.get(sample, "Unknown")
        #sample_counts_per_cell_type_per_species[species][cell_type] += 1



import re

import re

def auto_split_label(label, max_length=40):

    extracted_name = re.search(r'.*?_GO\.\d{7}\.(.*?)\.[A-Z]{2}$', label)
    if extracted_name:
        # Replace dots with spaces and split into words
        words = extracted_name.group(1).replace('.', ' ').split()
    else:
        # If no match, return an empty string or some placeholder
        return ""

    # Start with the first word and check length as adding more words
    split_label = words[0]
    current_length = len(words[0])

    # Add each word to the line until the max_length is reached, then start a new line
    for word in words[1:]:
        if current_length + len(word) + 1 <= max_length:
            split_label += ' ' + word
            current_length += len(word) + 1
        else:
            split_label += '\n' + word
            current_length = len(word)

    return split_label


# Provided cell types and colors
cell_colors = {
    "DCT_CNT_CD": "#3182bd",
    "DTL_ATL": "#fdd0a2",
    "EC": "seagreen",
    "IC": "orange",
    "Immune": "#c7e9c0",
    "Podo": "#000004",
    "Stromal": "limegreen",
    "PEC": "#fde725",
    "PT": "darkorchid",
    "TAL": "lightcoral",
}

# Function to extract cell type from feature name
def extract_cell_type(feature_name):
    for cell_type in cell_colors.keys():
        if feature_name.startswith(cell_type):
            return cell_type
    return "Unknown"

# Sample data - replace with your actual data

top_20_features = combined_pval.loc[sample_of_interest].nsmallest(10)
neg_log_pvals = -np.log10(top_20_features)
r2_values = combined_r2.loc[sample_of_interest][top_20_features.index]

# Create a reversed red colormap for R2 values
cmap = plt.cm.Reds_r  # '_r' suffix to reverse the colormap

norm = mcolors.Normalize(vmin=0, vmax=1)  # Normalization fixed from 0 to 1

# Assuming 'significant_counts_df' is your DataFrame and it has a 'GeneSet' and 'NormalizedPercentage' column
scale_factor = 4
base_size = 100
#base_size = 20

# Map features to their normalized percentage
feature_to_percentage = dict(zip(significant_counts_df['GeneSet'], significant_counts_df['NormalizedPercentage']))

# Create the dot plot
fig, ax = plt.subplots(figsize=(16, 5))
for i, (feature, neg_log_pval) in enumerate(neg_log_pvals.items()):
    cell_type = extract_cell_type(feature)
    bar_color = cell_colors.get(cell_type, 'grey')
    dot_color = cmap(norm(r2_values[feature]))
    
    # Get the normalized percentage for the dot size
    normalized_percentage = feature_to_percentage.get(feature, 0)
    dot_size = normalized_percentage * scale_factor + base_size

    ax.plot([0, neg_log_pval - (neg_log_pval / 10)], [i, i], color=bar_color, linewidth=3)
    ax.scatter(neg_log_pval, i, s=dot_size, color=dot_color, edgecolor='black', alpha=0.7, linewidth=0.5)

ax.set_yticks(range(len(top_20_features.index)))
ax.set_yticklabels(top_20_features.index, fontsize=10)
ax.set_xlabel('-log10(p-value)', fontsize=12)
ax.set_ylabel('Features', fontsize=12)
ax.set_title(f'Top Features for Sample {sample_of_interest}', fontsize=14)

# Invert y-axis to have the most significant features at the top
ax.invert_yaxis()

# Create a colorbar for the R2 values
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax, label='Scaled R2 Values', orientation='vertical')

# Legend for cell types
cell_legend_elements = [plt.Line2D([0], [0], color=color, lw=4, label=cell_type) for cell_type, color in cell_colors.items()]
legend1 = ax.legend(handles=cell_legend_elements, title='Cell Types', bbox_to_anchor=(1.5, 1), loc='upper left')

# Example normalized percentages for the dot size legend
example_percentages = [20, 40, 60, 80]
legend_dot_sizes = [p * scale_factor + base_size for p in example_percentages]

# Add legend for dot sizes
for p, size in zip(example_percentages, legend_dot_sizes):
    ax.scatter([], [], s=size, color='gray', edgecolor='black', alpha=0.7, label=f'{p}%')

legend2 = ax.legend(title='Frequency', bbox_to_anchor=(1.5, 0.2), loc='center left')
ax.add_artist(legend1)  # Add back the first legend

plt.tight_layout()
plt.show()









# Create the dot plot
fig, ax = plt.subplots(figsize=(10, 5))
for i, (feature, neg_log_pval) in enumerate(neg_log_pvals.items()):
    cell_type = extract_cell_type(feature)
    bar_color = cell_colors.get(cell_type, 'grey')
    
    # Draw the bar for significance
    ax.plot([0, neg_log_pval], [i, i], color=bar_color, linewidth=3)


# Process each feature name through auto_split_label and set as y-tick labels
processed_labels = [auto_split_label(feat) for feat in top_20_features.index]
ax.set_yticks(range(len(top_20_features.index)))
ax.set_yticklabels(processed_labels, fontsize=12)

#ax.set_yticklabels(top_20_features.index, fontsize=20)
ax.set_xlabel('-log10(p-value)', fontsize=12)
ax.set_ylabel('Features', fontsize=12)

# Invert y-axis to have the most significant features at the top
ax.invert_yaxis()

plt.tight_layout()
plt.show()









# Define your desired order of cell types
desired_order = [
     
"PT", 
"TAL", 
"DCT_CNT_CD", 
"IC",    
"EC",
"Stromal",      
"Immune",   
"DTL_ATL",  
"PEC",           
"Podo",    

    
]  # Replace with actual cell types

#top_20_features = combined_pval.loc[sample_of_interest].nsmallest(100)

sample_features = combined_pval.loc[sample_of_interest]

top_20_features = sample_features[sample_features < p_value_threshold]

# Assume 'combined_pval' is a DataFrame you have that contains the p-values

# Select the top 50 gene sets from the top features
top_gene_sets = top_20_features.index

# Group the top gene sets by cell type
grouped_data = pd.Series(top_gene_sets).apply(extract_cell_type).value_counts()

categories = list(grouped_data.index)

# Initialize ordered values with zero for all categories
ordered_values = [grouped_data.get(ct, 0) for ct in desired_order]

# Since we need to repeat the first value to close the circular graph
ordered_values += ordered_values[:1]

# Calculate the angle of each axis in the plot
N = len(desired_order)
angles = [n / float(N) * 2 * np.pi for n in range(N)]
angles += angles[:1]

# Initialize the spider plot
fig, ax = plt.subplots(figsize=(4, 4), subplot_kw=dict(polar=True))

# Draw one axis per variable and add labels
plt.xticks(angles[:-1], desired_order)

# Draw ylabels and set plot limits
ax.set_rlabel_position(0)
plt.yticks(color="grey", size=5)
#plt.ylim(0, 1200)


# Neutral color for shading
shade_color = 'lightgrey'

# Fill the entire area under the radar chart with a neutral color
ax.fill(angles, ordered_values, shade_color, alpha=0.9)

# Draw thin lines connecting the points
ax.plot(angles, ordered_values, color='grey', linewidth=1, linestyle='-', alpha=0.8)

# Plot each line segment in its respective cell type color
for idx in range(N):
    color = cell_colors.get(desired_order[idx], "grey")
    ax.plot([angles[idx], 0], [ordered_values[idx], 0], color=color, linewidth=2)


#########THIS WAS FIXED############
# List to store cell types with all NaN values
missing_samples = []

# Loop through each cell type
for celltype in desired_order:
    # Subset 'allfeatures' to only include features starting with the current cell type
    celltype_features = sample_features[sample_features.index.str.startswith(celltype)]
    
    # Check if the number of NaN values is above the threshold
    if celltype_features.isna().sum() > nan_threshold:
        # Add the celltype to the missing_samples list
        missing_samples.append(celltype)

for idx, label in enumerate(ax.get_xticklabels()):
    # Check if the cell type is in the missing_samples list
    if desired_order[idx] in missing_samples:
        label.set_color('red')

# Add a title
plt.title('Cell Type Distribution in Top Gene Sets for Sample ' + sample_of_interest, size=11, y=1.1)

plt.show()


# Desired cell type for visualization
desired_cell_type = ct_plot1

# Assuming 'combined_pval' and 'combined_r2' are DataFrames with samples as rows and features as columns
# Filter for the specific sample of interest
sample_pvals = combined_pval.loc[sample_of_interest]
sample_r2 = combined_r2.loc[sample_of_interest]

# Filter the top features for the specific sample and cell type
#top_features_for_sample = sample_pvals.nsmallest(25)
filtered_features = sample_pvals[sample_pvals.index.map(extract_cell_type) == desired_cell_type].nsmallest(20)
filtered_neg_log_pvals = -np.log10(filtered_features)
filtered_r2_values = sample_r2[filtered_features.index]

# Now, update the dot plot code to use the filtered data
fig, ax = plt.subplots(figsize=(16, 5))
for i, (feature, neg_log_pval) in enumerate(filtered_neg_log_pvals.items()):
    cell_type = extract_cell_type(feature)
    bar_color = cell_colors.get(cell_type, 'grey')
    dot_color = cmap(norm(filtered_r2_values[feature]))
    
    # Get the normalized percentage for the dot size (if applicable)
    normalized_percentage = feature_to_percentage.get(feature, 0)
    dot_size = normalized_percentage * scale_factor + base_size

    ax.plot([0, neg_log_pval - (neg_log_pval / 10)], [i, i], color=bar_color, linewidth=3)
    ax.scatter(neg_log_pval, i, s=dot_size, color=dot_color, edgecolor='black', alpha=0.7, linewidth=0.5)

ax.set_yticks(range(len(filtered_features.index)))

prefix_to_remove = f"{desired_cell_type}_"
cleaned_labels = [label.replace(prefix_to_remove, '', 1) for label in filtered_features.index]

# Apply the auto-splitting to each label
cleaned_labels = [auto_split_label(label) for label in cleaned_labels]

ax.set_yticklabels(cleaned_labels, fontsize=10)
ax.set_xlabel('-log10(p-value)', fontsize=15)
#ax.set_ylabel('Features', fontsize=12)
#ax.set_title(f'Top Features for Sample {sample_of_interest} - Cell Type: {desired_cell_type}', fontsize=14)
ax.invert_yaxis()

# Create a colorbar for the R2 values
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax, label='Scaled R2 Values', orientation='vertical')

# Legend for cell types
#cell_legend_elements = [plt.Line2D([0], [0], color=color, lw=4, label=cell_type) for cell_type, color in cell_colors.items()]
#legend1 = ax.legend(handles=cell_legend_elements, title='Cell Types', bbox_to_anchor=(1.5, 1), loc='upper left')

# Example normalized percentages for the dot size legend
example_percentages = [20, 40, 60, 80]
legend_dot_sizes = [p * scale_factor + base_size for p in example_percentages]

# Add legend for dot sizes
for p, size in zip(example_percentages, legend_dot_sizes):
    ax.scatter([], [], s=size, color='gray', edgecolor='black', alpha=0.7, label=f'{p}%')

legend2 = ax.legend(title='Frequency', bbox_to_anchor=(1.5, 0.2), loc='center left', fontsize='large')
#ax.add_artist(legend1)  # Add back the first legend

plt.tight_layout()
plt.show()

In [ ]:
sample_of_interest = 'KRAD70'

#max_value = 300

ct_plot1 = "DCT_CNT_CD"

ct_plot2 = "PEC"

ct_plot3 = "PT"


# Replace with your actual file paths and cell types
cell_types = ["Podo", "TAL", "DCT_CNT_CD", "EC", "Stromal", "Immune", "PEC", "PT", "IC", "DTL_ATL"] # etc.
# Paths as specified in your first script
base_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/human_extended/output_human_extended/"
r2_folder = "R2_GO_unfiltered/"
pval_folder = "Padj_GO_unfiltered/"
metadat_path = "/home/isilon/users/o_kloetzer/Atlas/scSPECTRA/R2_pval/Atlas_Extended_II_Albuminuria_gpt.csv"  # Update with the actual path to 'metadat'

def extract_cell_type(gene_set):
    for cell_type in cell_types:
        if gene_set.startswith(cell_type):
            return cell_type
    return "Unknown"  # or some default value

# Read metadata
metadat = pd.read_csv(metadat_path)
unique_samples = metadat[metadat['Disease_level2'] != "Control"]['Sample'].unique()
#unique_samples = metadat[(metadat['Disease_level2'] != "Control") & (metadat['Project/Dataset'] == "cal_CKD")]['Sample'].unique()

# Initialize combined dataframes for R2 and p-values
combined_r2 = pd.DataFrame(index=unique_samples)
combined_pval = pd.DataFrame(index=unique_samples)
sample_counts_per_cell_type = {}

# Process each cell type
for cell_type in cell_types:
    r2_path = os.path.join(base_path, r2_folder, f"R2_{cell_type}.csv")
    pval_path = os.path.join(base_path, pval_folder, f"Pval_{cell_type}.csv")
    
    r2_df = pd.read_csv(r2_path, index_col=0)
    pval_df = pd.read_csv(pval_path, index_col=0)
    
    # Modify column names
    r2_df.columns = [f"{cell_type}_{col}" for col in r2_df.columns]
    pval_df.columns = [f"{cell_type}_{col}" for col in pval_df.columns]
    
    # Merge data
    combined_r2 = combined_r2.join(r2_df, how='left')
    combined_pval = combined_pval.join(pval_df, how='left')

    # Count samples per cell type
    r2_df = r2_df[r2_df.index.isin(unique_samples)]
    sample_counts_per_cell_type[cell_type] = r2_df.shape[0]

# Handle NA values in p-values
combined_pval.fillna(1, inplace=True)

print(combined_r2)

# Initialize a MinMaxScaler
min_max_scaler = MinMaxScaler()

# Scale the combined_r2 data
# Note: The fit_transform method expects a 2D array, so we use .values to convert the DataFrame
min_max_scaled_r2_values = min_max_scaler.fit_transform(combined_r2.values)

# Create a new DataFrame from the min-max scaled data, preserving the index and column names

combined_r2 = pd.DataFrame(min_max_scaled_r2_values, index=combined_r2.index, columns=combined_r2.columns)

# Set p-value threshold
p_value_threshold = 0.05

# Identify significant gene sets
significant_matrix = combined_pval < p_value_threshold

# Count significant occurrences
significant_counts = significant_matrix.sum()

# Create DataFrame for counts and cell types
significant_counts_df = pd.DataFrame({'GeneSet': significant_counts.index, 'Count': significant_counts.values})

# Apply the function to extract the full cell type name
significant_counts_df['CellType'] = significant_counts_df['GeneSet'].apply(extract_cell_type)

# Normalize by sample number per cell type
significant_counts_df['NormalizedPercentage'] = significant_counts_df.apply(
    lambda row: (row['Count'] / sample_counts_per_cell_type.get(row['CellType'], 1)) * 100, axis=1
)

# Display final DataFrame
print(significant_counts_df.head())  # You can modify this to display as needed

sorted_df = significant_counts_df.sort_values(by='NormalizedPercentage', ascending=False)
display(sorted_df)

import re

def auto_split_label(label, max_length=40):
    # Split the label into words
    words = label.split()

    # Function to remove suffix
    def remove_suffix(word):
        return re.sub(r'\(GO:\d{7}\)$', '', word)

    # Start with the first word (with suffix removed)
    first_word = remove_suffix(words[0])
    split_label = first_word
    current_length = len(first_word)

    # Add each word to the line until the max_length is reached, then start a new line
    for word in words[1:]:
        word = remove_suffix(word)
        if current_length + len(word) + 1 <= max_length:
            split_label += ' ' + word
            current_length += len(word) + 1
        else:
            split_label += '\n' + word
            current_length = len(word)

    return split_label


# Provided cell types and colors
cell_colors = {
    "DCT_CNT_CD": "#3182bd",
    "DTL_ATL": "#fdd0a2",
    "EC": "seagreen",
    "IC": "orange",
    "Immune": "#c7e9c0",
    "Podo": "#000004",
    "Stromal": "limegreen",
    "PEC": "#fde725",
    "PT": "darkorchid",
    "TAL": "lightcoral",
}

# Function to extract cell type from feature name
def extract_cell_type(feature_name):
    for cell_type in cell_colors.keys():
        if feature_name.startswith(cell_type):
            return cell_type
    return "Unknown"

# Sample data - replace with your actual data

top_20_features = combined_pval.loc[sample_of_interest].nsmallest(25)
neg_log_pvals = -np.log10(top_20_features)
r2_values = combined_r2.loc[sample_of_interest][top_20_features.index]

# Create a reversed red colormap for R2 values
cmap = plt.cm.Reds_r  # '_r' suffix to reverse the colormap

norm = mcolors.Normalize(vmin=0, vmax=1)  # Normalization fixed from 0 to 1

# Assuming 'significant_counts_df' is your DataFrame and it has a 'GeneSet' and 'NormalizedPercentage' column
scale_factor = 4
base_size = 100
#base_size = 20

# Map features to their normalized percentage
feature_to_percentage = dict(zip(significant_counts_df['GeneSet'], significant_counts_df['NormalizedPercentage']))

# Create the dot plot
fig, ax = plt.subplots(figsize=(16, 5))
for i, (feature, neg_log_pval) in enumerate(neg_log_pvals.items()):
    cell_type = extract_cell_type(feature)
    bar_color = cell_colors.get(cell_type, 'grey')
    dot_color = cmap(norm(r2_values[feature]))
    
    # Get the normalized percentage for the dot size
    normalized_percentage = feature_to_percentage.get(feature, 0)
    dot_size = normalized_percentage * scale_factor + base_size

    ax.plot([0, neg_log_pval - (neg_log_pval / 10)], [i, i], color=bar_color, linewidth=3)
    ax.scatter(neg_log_pval, i, s=dot_size, color=dot_color, edgecolor='black', alpha=0.7, linewidth=0.5)

ax.set_yticks(range(len(top_20_features.index)))
ax.set_yticklabels(top_20_features.index, fontsize=10)
ax.set_xlabel('-log10(p-value)', fontsize=12)
ax.set_ylabel('Features', fontsize=12)
ax.set_title(f'Top Features for Sample {sample_of_interest}', fontsize=14)

# Invert y-axis to have the most significant features at the top
ax.invert_yaxis()

# Create a colorbar for the R2 values
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax, label='Scaled R2 Values', orientation='vertical')

# Legend for cell types
cell_legend_elements = [plt.Line2D([0], [0], color=color, lw=4, label=cell_type) for cell_type, color in cell_colors.items()]
legend1 = ax.legend(handles=cell_legend_elements, title='Cell Types', bbox_to_anchor=(1.5, 1), loc='upper left')

# Example normalized percentages for the dot size legend
example_percentages = [20, 40, 60, 80]
legend_dot_sizes = [p * scale_factor + base_size for p in example_percentages]

# Add legend for dot sizes
for p, size in zip(example_percentages, legend_dot_sizes):
    ax.scatter([], [], s=size, color='gray', edgecolor='black', alpha=0.7, label=f'{p}%')

legend2 = ax.legend(title='Frequency', bbox_to_anchor=(1.5, 0.2), loc='center left')
ax.add_artist(legend1)  # Add back the first legend

plt.tight_layout()
plt.show()


# Define your desired order of cell types
desired_order = [
     
"PT", 
"TAL", 
"DCT_CNT_CD", 
"IC",    
"EC",
"Stromal",      
"Immune",   
"DTL_ATL",  
"PEC",           
"Podo",    

    
]  # Replace with actual cell types

#top_20_features = combined_pval.loc[sample_of_interest].nsmallest(100)

sample_features = combined_pval.loc[sample_of_interest]

top_20_features = sample_features[sample_features < p_value_threshold]

# Assume 'combined_pval' is a DataFrame you have that contains the p-values

# Select the top 50 gene sets from the top features
top_gene_sets = top_20_features.index

# Group the top gene sets by cell type
grouped_data = pd.Series(top_gene_sets).apply(extract_cell_type).value_counts()

categories = list(grouped_data.index)

# Initialize ordered values with zero for all categories
ordered_values = [grouped_data.get(ct, 0) for ct in desired_order]

# Since we need to repeat the first value to close the circular graph
ordered_values += ordered_values[:1]

# Calculate the angle of each axis in the plot
N = len(desired_order)
angles = [n / float(N) * 2 * np.pi for n in range(N)]
angles += angles[:1]

# Initialize the spider plot
fig, ax = plt.subplots(figsize=(4, 4), subplot_kw=dict(polar=True))

# Draw one axis per variable and add labels
plt.xticks(angles[:-1], desired_order)

# Draw ylabels and set plot limits
ax.set_rlabel_position(0)
plt.yticks(color="grey", size=5)


#plt.ylim(0, max_value)


# Neutral color for shading
shade_color = 'lightgrey'

# Fill the entire area under the radar chart with a neutral color
ax.fill(angles, ordered_values, shade_color, alpha=0.9)

# Draw thin lines connecting the points
ax.plot(angles, ordered_values, color='grey', linewidth=1, linestyle='-', alpha=0.8)

# Plot each line segment in its respective cell type color
for idx in range(N):
    color = cell_colors.get(desired_order[idx], "grey")
    ax.plot([angles[idx], 0], [ordered_values[idx], 0], color=color, linewidth=2)

# Change the color of tick labels to red if the cell type is missing
for idx, label in enumerate(ax.get_xticklabels()):
    if desired_order[idx] not in categories:
        label.set_color('red')

# Add a title
plt.title('Cell Type Distribution in Top Gene Sets for Sample ' + sample_of_interest, size=11, y=1.1)

plt.show()


# Desired cell type for visualization
desired_cell_type = ct_plot1

# Assuming 'combined_pval' and 'combined_r2' are DataFrames with samples as rows and features as columns
# Filter for the specific sample of interest
sample_pvals = combined_pval.loc[sample_of_interest]
sample_r2 = combined_r2.loc[sample_of_interest]

# Filter the top features for the specific sample and cell type
#top_features_for_sample = sample_pvals.nsmallest(25)
filtered_features = sample_pvals[sample_pvals.index.map(extract_cell_type) == desired_cell_type].nsmallest(5)
filtered_neg_log_pvals = -np.log10(filtered_features)
filtered_r2_values = sample_r2[filtered_features.index]

# Now, update the dot plot code to use the filtered data
fig, ax = plt.subplots(figsize=(16, 5))
for i, (feature, neg_log_pval) in enumerate(filtered_neg_log_pvals.items()):
    cell_type = extract_cell_type(feature)
    bar_color = cell_colors.get(cell_type, 'grey')
    dot_color = cmap(norm(filtered_r2_values[feature]))
    
    # Get the normalized percentage for the dot size (if applicable)
    normalized_percentage = feature_to_percentage.get(feature, 0)
    dot_size = normalized_percentage * scale_factor + base_size

    ax.plot([0, neg_log_pval - (neg_log_pval / 10)], [i, i], color=bar_color, linewidth=3)
    ax.scatter(neg_log_pval, i, s=dot_size, color=dot_color, edgecolor='black', alpha=0.7, linewidth=0.5)

ax.set_yticks(range(len(filtered_features.index)))

prefix_to_remove = f"{desired_cell_type}_"
cleaned_labels = [label.replace(prefix_to_remove, '', 1) for label in filtered_features.index]

# Apply the auto-splitting to each label
cleaned_labels = [auto_split_label(label) for label in cleaned_labels]

ax.set_yticklabels(cleaned_labels, fontsize=22)
ax.set_xlabel('-log10(p-value)', fontsize=15)
#ax.set_ylabel('Features', fontsize=12)
#ax.set_title(f'Top Features for Sample {sample_of_interest} - Cell Type: {desired_cell_type}', fontsize=14)
ax.invert_yaxis()

# Create a colorbar for the R2 values
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax, label='Scaled R2 Values', orientation='vertical')

# Legend for cell types
#cell_legend_elements = [plt.Line2D([0], [0], color=color, lw=4, label=cell_type) for cell_type, color in cell_colors.items()]
#legend1 = ax.legend(handles=cell_legend_elements, title='Cell Types', bbox_to_anchor=(1.5, 1), loc='upper left')

# Example normalized percentages for the dot size legend
example_percentages = [20, 40, 60, 80]
legend_dot_sizes = [p * scale_factor + base_size for p in example_percentages]

# Add legend for dot sizes
for p, size in zip(example_percentages, legend_dot_sizes):
    ax.scatter([], [], s=size, color='gray', edgecolor='black', alpha=0.7, label=f'{p}%')

legend2 = ax.legend(title='Frequency', bbox_to_anchor=(1.5, 0.2), loc='center left', fontsize='large')
#ax.add_artist(legend1)  # Add back the first legend

plt.tight_layout()
plt.show()




# Desired cell type for visualization
desired_cell_type = ct_plot2

# Assuming 'combined_pval' and 'combined_r2' are DataFrames with samples as rows and features as columns
# Filter for the specific sample of interest
sample_pvals = combined_pval.loc[sample_of_interest]
sample_r2 = combined_r2.loc[sample_of_interest]

# Filter the top features for the specific sample and cell type
#top_features_for_sample = sample_pvals.nsmallest(25)
filtered_features = sample_pvals[sample_pvals.index.map(extract_cell_type) == desired_cell_type].nsmallest(5)
filtered_neg_log_pvals = -np.log10(filtered_features)
filtered_r2_values = sample_r2[filtered_features.index]

# Now, update the dot plot code to use the filtered data
fig, ax = plt.subplots(figsize=(16, 5))
for i, (feature, neg_log_pval) in enumerate(filtered_neg_log_pvals.items()):
    cell_type = extract_cell_type(feature)
    bar_color = cell_colors.get(cell_type, 'grey')
    dot_color = cmap(norm(filtered_r2_values[feature]))
    
    # Get the normalized percentage for the dot size (if applicable)
    normalized_percentage = feature_to_percentage.get(feature, 0)
    dot_size = normalized_percentage * scale_factor + base_size

    ax.plot([0, neg_log_pval - (neg_log_pval / 10)], [i, i], color=bar_color, linewidth=3)
    ax.scatter(neg_log_pval, i, s=dot_size, color=dot_color, edgecolor='black', alpha=0.7, linewidth=0.5)

ax.set_yticks(range(len(filtered_features.index)))

prefix_to_remove = f"{desired_cell_type}_"
cleaned_labels = [label.replace(prefix_to_remove, '', 1) for label in filtered_features.index]

# Apply the auto-splitting to each label
cleaned_labels = [auto_split_label(label) for label in cleaned_labels]

ax.set_yticklabels(cleaned_labels, fontsize=22)
ax.set_xlabel('-log10(p-value)', fontsize=15)
#ax.set_ylabel('Features', fontsize=12)
#ax.set_title(f'Top Features for Sample {sample_of_interest} - Cell Type: {desired_cell_type}', fontsize=14)
ax.invert_yaxis()

# Create a colorbar for the R2 values
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax, label='Scaled R2 Values', orientation='vertical')

# Legend for cell types
#cell_legend_elements = [plt.Line2D([0], [0], color=color, lw=4, label=cell_type) for cell_type, color in cell_colors.items()]
#legend1 = ax.legend(handles=cell_legend_elements, title='Cell Types', bbox_to_anchor=(1.5, 1), loc='upper left')

# Example normalized percentages for the dot size legend
example_percentages = [20, 40, 60, 80]
legend_dot_sizes = [p * scale_factor + base_size for p in example_percentages]

# Add legend for dot sizes
for p, size in zip(example_percentages, legend_dot_sizes):
    ax.scatter([], [], s=size, color='gray', edgecolor='black', alpha=0.7, label=f'{p}%')

legend2 = ax.legend(title='Frequency', bbox_to_anchor=(1.5, 0.2), loc='center left', fontsize='large')
#ax.add_artist(legend1)  # Add back the first legend

plt.tight_layout()
plt.show()





# Desired cell type for visualization
desired_cell_type = ct_plot3

# Assuming 'combined_pval' and 'combined_r2' are DataFrames with samples as rows and features as columns
# Filter for the specific sample of interest
sample_pvals = combined_pval.loc[sample_of_interest]
sample_r2 = combined_r2.loc[sample_of_interest]

# Filter the top features for the specific sample and cell type
#top_features_for_sample = sample_pvals.nsmallest(25)
filtered_features = sample_pvals[sample_pvals.index.map(extract_cell_type) == desired_cell_type].nsmallest(5)
filtered_neg_log_pvals = -np.log10(filtered_features)
filtered_r2_values = sample_r2[filtered_features.index]

# Now, update the dot plot code to use the filtered data
fig, ax = plt.subplots(figsize=(16, 5))
for i, (feature, neg_log_pval) in enumerate(filtered_neg_log_pvals.items()):
    cell_type = extract_cell_type(feature)
    bar_color = cell_colors.get(cell_type, 'grey')
    dot_color = cmap(norm(filtered_r2_values[feature]))
    
    # Get the normalized percentage for the dot size (if applicable)
    normalized_percentage = feature_to_percentage.get(feature, 0)
    dot_size = normalized_percentage * scale_factor + base_size

    ax.plot([0, neg_log_pval - (neg_log_pval / 10)], [i, i], color=bar_color, linewidth=3)
    ax.scatter(neg_log_pval, i, s=dot_size, color=dot_color, edgecolor='black', alpha=0.7, linewidth=0.5)

ax.set_yticks(range(len(filtered_features.index)))

prefix_to_remove = f"{desired_cell_type}_"
cleaned_labels = [label.replace(prefix_to_remove, '', 1) for label in filtered_features.index]

# Apply the auto-splitting to each label
cleaned_labels = [auto_split_label(label) for label in cleaned_labels]

ax.set_yticklabels(cleaned_labels, fontsize=22)
ax.set_xlabel('-log10(p-value)', fontsize=15)
#ax.set_ylabel('Features', fontsize=12)
#ax.set_title(f'Top Features for Sample {sample_of_interest} - Cell Type: {desired_cell_type}', fontsize=14)
ax.invert_yaxis()

# Create a colorbar for the R2 values
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax, label='Scaled R2 Values', orientation='vertical')

# Legend for cell types
#cell_legend_elements = [plt.Line2D([0], [0], color=color, lw=4, label=cell_type) for cell_type, color in cell_colors.items()]
#legend1 = ax.legend(handles=cell_legend_elements, title='Cell Types', bbox_to_anchor=(1.5, 1), loc='upper left')

# Example normalized percentages for the dot size legend
example_percentages = [20, 40, 60, 80]
legend_dot_sizes = [p * scale_factor + base_size for p in example_percentages]

# Add legend for dot sizes
for p, size in zip(example_percentages, legend_dot_sizes):
    ax.scatter([], [], s=size, color='gray', edgecolor='black', alpha=0.7, label=f'{p}%')

legend2 = ax.legend(title='Frequency', bbox_to_anchor=(1.5, 0.2), loc='center left', fontsize='large')
#ax.add_artist(legend1)  # Add back the first legend

plt.tight_layout()
plt.show()

In [ ]:
p_value_threshold = 0.05

sample_of_interest = "PKD4_humphreys_ADPKD"

#max_value = 300

ct_plot1 = "PT"

#specify how many nan we tolerate (critical if applying gene filtering during spectra analysis)
nan_threshold = 1000

import pandas as pd
import os

# Define cell types and file paths
cell_types = ["PT", "Podo", "TAL", "DCT_CNT_CD", "EC", "Stromal", "IC", "Immune", "PEC", "DTL_ATL"]

#cell_types = ["PEC", "Immune"]

base_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/human_extended/output_human_extended/"
r2_folder = "R2_GO_unfiltered/"
pval_folder = "Padj_GO_unfiltered/"
metadat_path = "/home/isilon/users/o_kloetzer/Atlas/scSPECTRA/R2_pval/Atlas_Extended_II_Albuminuria_gpt.csv"  # Update with the actual path to 'metadat'
metadata_path = metadat_path

def extract_cell_type(gene_set):
    for cell_type in cell_types:
        if gene_set.startswith(cell_type):
            return cell_type
    return "Unknown"

# Read metadata
metadata = pd.read_csv(metadata_path)
unique_samples = metadata[metadata['Disease_level1'] != "Control"]['Sample'].unique()
#sample_to_species = metadata.set_index('orig_ident')['species'].to_dict()

# Initialize combined dataframes for R2 and p-values
combined_r2 = pd.DataFrame(index=unique_samples)
combined_pval = pd.DataFrame(index=unique_samples)
sample_counts_per_cell_type = {}
#sample_counts_per_cell_type_per_species = {species: {ct: 0 for ct in cell_types} for species in metadata['species'].unique()}

# Process each cell type
for cell_type in cell_types:
    r2_path = os.path.join(base_path, r2_folder, f"R2_{cell_type}.csv")
    pval_path = os.path.join(base_path, pval_folder, f"Pval_{cell_type}.csv")
    
    r2_df = pd.read_csv(r2_path, index_col=0)
    r2_df = r2_df[r2_df.index.isin(unique_samples)]
    pval_df = pd.read_csv(pval_path, index_col=0)
    pval_df = pval_df[pval_df.index.isin(unique_samples)]
    
    r2_df.columns = [f"{cell_type}_{col}" for col in r2_df.columns]
    pval_df.columns = [f"{cell_type}_{col}" for col in pval_df.columns]
    
    combined_r2 = combined_r2.join(r2_df, how='left')
    combined_pval = combined_pval.join(pval_df, how='left')

    sample_counts_per_cell_type[cell_type] = r2_df.shape[0]
    #for sample in r2_df.index:
        #species = sample_to_species.get(sample, "Unknown")
        #sample_counts_per_cell_type_per_species[species][cell_type] += 1



import re

import re

def auto_split_label(label, max_length=40):

    extracted_name = re.search(r'.*?_GO\.\d{7}\.(.*?)\.[A-Z]{2}$', label)
    if extracted_name:
        # Replace dots with spaces and split into words
        words = extracted_name.group(1).replace('.', ' ').split()
    else:
        # If no match, return an empty string or some placeholder
        return ""

    # Start with the first word and check length as adding more words
    split_label = words[0]
    current_length = len(words[0])

    # Add each word to the line until the max_length is reached, then start a new line
    for word in words[1:]:
        if current_length + len(word) + 1 <= max_length:
            split_label += ' ' + word
            current_length += len(word) + 1
        else:
            split_label += '\n' + word
            current_length = len(word)

    return split_label


# Provided cell types and colors
cell_colors = {
    "DCT_CNT_CD": "#3182bd",
    "DTL_ATL": "#fdd0a2",
    "EC": "seagreen",
    "IC": "orange",
    "Immune": "#c7e9c0",
    "Podo": "#000004",
    "Stromal": "limegreen",
    "PEC": "#fde725",
    "PT": "darkorchid",
    "TAL": "lightcoral",
}

# Function to extract cell type from feature name
def extract_cell_type(feature_name):
    for cell_type in cell_colors.keys():
        if feature_name.startswith(cell_type):
            return cell_type
    return "Unknown"

# Sample data - replace with your actual data

top_20_features = combined_pval.loc[sample_of_interest].nsmallest(10)
neg_log_pvals = -np.log10(top_20_features)
r2_values = combined_r2.loc[sample_of_interest][top_20_features.index]

# Create a reversed red colormap for R2 values
cmap = plt.cm.Reds_r  # '_r' suffix to reverse the colormap

norm = mcolors.Normalize(vmin=0, vmax=1)  # Normalization fixed from 0 to 1

# Assuming 'significant_counts_df' is your DataFrame and it has a 'GeneSet' and 'NormalizedPercentage' column
scale_factor = 4
base_size = 100
#base_size = 20

# Map features to their normalized percentage
feature_to_percentage = dict(zip(significant_counts_df['GeneSet'], significant_counts_df['NormalizedPercentage']))

# Create the dot plot
fig, ax = plt.subplots(figsize=(16, 5))
for i, (feature, neg_log_pval) in enumerate(neg_log_pvals.items()):
    cell_type = extract_cell_type(feature)
    bar_color = cell_colors.get(cell_type, 'grey')
    dot_color = cmap(norm(r2_values[feature]))
    
    # Get the normalized percentage for the dot size
    normalized_percentage = feature_to_percentage.get(feature, 0)
    dot_size = normalized_percentage * scale_factor + base_size

    ax.plot([0, neg_log_pval - (neg_log_pval / 10)], [i, i], color=bar_color, linewidth=3)
    ax.scatter(neg_log_pval, i, s=dot_size, color=dot_color, edgecolor='black', alpha=0.7, linewidth=0.5)

ax.set_yticks(range(len(top_20_features.index)))
ax.set_yticklabels(top_20_features.index, fontsize=10)
ax.set_xlabel('-log10(p-value)', fontsize=12)
ax.set_ylabel('Features', fontsize=12)
ax.set_title(f'Top Features for Sample {sample_of_interest}', fontsize=14)

# Invert y-axis to have the most significant features at the top
ax.invert_yaxis()

# Create a colorbar for the R2 values
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax, label='Scaled R2 Values', orientation='vertical')

# Legend for cell types
cell_legend_elements = [plt.Line2D([0], [0], color=color, lw=4, label=cell_type) for cell_type, color in cell_colors.items()]
legend1 = ax.legend(handles=cell_legend_elements, title='Cell Types', bbox_to_anchor=(1.5, 1), loc='upper left')

# Example normalized percentages for the dot size legend
example_percentages = [20, 40, 60, 80]
legend_dot_sizes = [p * scale_factor + base_size for p in example_percentages]

# Add legend for dot sizes
for p, size in zip(example_percentages, legend_dot_sizes):
    ax.scatter([], [], s=size, color='gray', edgecolor='black', alpha=0.7, label=f'{p}%')

legend2 = ax.legend(title='Frequency', bbox_to_anchor=(1.5, 0.2), loc='center left')
ax.add_artist(legend1)  # Add back the first legend

plt.tight_layout()
plt.show()









# Create the dot plot
fig, ax = plt.subplots(figsize=(10, 5))
for i, (feature, neg_log_pval) in enumerate(neg_log_pvals.items()):
    cell_type = extract_cell_type(feature)
    bar_color = cell_colors.get(cell_type, 'grey')
    
    # Draw the bar for significance
    ax.plot([0, neg_log_pval], [i, i], color=bar_color, linewidth=3)


# Process each feature name through auto_split_label and set as y-tick labels
processed_labels = [auto_split_label(feat) for feat in top_20_features.index]
ax.set_yticks(range(len(top_20_features.index)))
ax.set_yticklabels(processed_labels, fontsize=12)

#ax.set_yticklabels(top_20_features.index, fontsize=20)
ax.set_xlabel('-log10(p-value)', fontsize=12)
ax.set_ylabel('Features', fontsize=12)

# Invert y-axis to have the most significant features at the top
ax.invert_yaxis()

plt.tight_layout()
plt.show()









# Define your desired order of cell types
desired_order = [
     
"PT", 
"TAL", 
"DCT_CNT_CD", 
"IC",    
"EC",
"Stromal",      
"Immune",   
"DTL_ATL",  
"PEC",           
"Podo",    

    
]  # Replace with actual cell types

#top_20_features = combined_pval.loc[sample_of_interest].nsmallest(100)

sample_features = combined_pval.loc[sample_of_interest]

top_20_features = sample_features[sample_features < p_value_threshold]

# Assume 'combined_pval' is a DataFrame you have that contains the p-values

# Select the top 50 gene sets from the top features
top_gene_sets = top_20_features.index

# Group the top gene sets by cell type
grouped_data = pd.Series(top_gene_sets).apply(extract_cell_type).value_counts()

categories = list(grouped_data.index)

# Initialize ordered values with zero for all categories
ordered_values = [grouped_data.get(ct, 0) for ct in desired_order]

# Since we need to repeat the first value to close the circular graph
ordered_values += ordered_values[:1]

# Calculate the angle of each axis in the plot
N = len(desired_order)
angles = [n / float(N) * 2 * np.pi for n in range(N)]
angles += angles[:1]

# Initialize the spider plot
fig, ax = plt.subplots(figsize=(4, 4), subplot_kw=dict(polar=True))

# Draw one axis per variable and add labels
plt.xticks(angles[:-1], desired_order)

# Draw ylabels and set plot limits
ax.set_rlabel_position(0)
plt.yticks(color="grey", size=5)
#plt.ylim(0, 1200)


# Neutral color for shading
shade_color = 'lightgrey'

# Fill the entire area under the radar chart with a neutral color
ax.fill(angles, ordered_values, shade_color, alpha=0.9)

# Draw thin lines connecting the points
ax.plot(angles, ordered_values, color='grey', linewidth=1, linestyle='-', alpha=0.8)

# Plot each line segment in its respective cell type color
for idx in range(N):
    color = cell_colors.get(desired_order[idx], "grey")
    ax.plot([angles[idx], 0], [ordered_values[idx], 0], color=color, linewidth=2)


#########THIS WAS FIXED############
# List to store cell types with all NaN values
missing_samples = []

# Loop through each cell type
for celltype in desired_order:
    # Subset 'allfeatures' to only include features starting with the current cell type
    celltype_features = sample_features[sample_features.index.str.startswith(celltype)]
    
    # Check if the number of NaN values is above the threshold
    if celltype_features.isna().sum() > nan_threshold:
        # Add the celltype to the missing_samples list
        missing_samples.append(celltype)

for idx, label in enumerate(ax.get_xticklabels()):
    # Check if the cell type is in the missing_samples list
    if desired_order[idx] in missing_samples:
        label.set_color('red')

# Add a title
plt.title('Cell Type Distribution in Top Gene Sets for Sample ' + sample_of_interest, size=11, y=1.1)

plt.show()


# Desired cell type for visualization
desired_cell_type = ct_plot1

# Assuming 'combined_pval' and 'combined_r2' are DataFrames with samples as rows and features as columns
# Filter for the specific sample of interest
sample_pvals = combined_pval.loc[sample_of_interest]
sample_r2 = combined_r2.loc[sample_of_interest]

# Filter the top features for the specific sample and cell type
#top_features_for_sample = sample_pvals.nsmallest(25)
filtered_features = sample_pvals[sample_pvals.index.map(extract_cell_type) == desired_cell_type].nsmallest(20)
filtered_neg_log_pvals = -np.log10(filtered_features)
filtered_r2_values = sample_r2[filtered_features.index]

# Now, update the dot plot code to use the filtered data
fig, ax = plt.subplots(figsize=(16, 5))
for i, (feature, neg_log_pval) in enumerate(filtered_neg_log_pvals.items()):
    cell_type = extract_cell_type(feature)
    bar_color = cell_colors.get(cell_type, 'grey')
    dot_color = cmap(norm(filtered_r2_values[feature]))
    
    # Get the normalized percentage for the dot size (if applicable)
    normalized_percentage = feature_to_percentage.get(feature, 0)
    dot_size = normalized_percentage * scale_factor + base_size

    ax.plot([0, neg_log_pval - (neg_log_pval / 10)], [i, i], color=bar_color, linewidth=3)
    ax.scatter(neg_log_pval, i, s=dot_size, color=dot_color, edgecolor='black', alpha=0.7, linewidth=0.5)

ax.set_yticks(range(len(filtered_features.index)))

prefix_to_remove = f"{desired_cell_type}_"
cleaned_labels = [label.replace(prefix_to_remove, '', 1) for label in filtered_features.index]

# Apply the auto-splitting to each label
cleaned_labels = [auto_split_label(label) for label in cleaned_labels]

ax.set_yticklabels(cleaned_labels, fontsize=10)
ax.set_xlabel('-log10(p-value)', fontsize=15)
#ax.set_ylabel('Features', fontsize=12)
#ax.set_title(f'Top Features for Sample {sample_of_interest} - Cell Type: {desired_cell_type}', fontsize=14)
ax.invert_yaxis()

# Create a colorbar for the R2 values
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax, label='Scaled R2 Values', orientation='vertical')

# Legend for cell types
#cell_legend_elements = [plt.Line2D([0], [0], color=color, lw=4, label=cell_type) for cell_type, color in cell_colors.items()]
#legend1 = ax.legend(handles=cell_legend_elements, title='Cell Types', bbox_to_anchor=(1.5, 1), loc='upper left')

# Example normalized percentages for the dot size legend
example_percentages = [20, 40, 60, 80]
legend_dot_sizes = [p * scale_factor + base_size for p in example_percentages]

# Add legend for dot sizes
for p, size in zip(example_percentages, legend_dot_sizes):
    ax.scatter([], [], s=size, color='gray', edgecolor='black', alpha=0.7, label=f'{p}%')

legend2 = ax.legend(title='Frequency', bbox_to_anchor=(1.5, 0.2), loc='center left', fontsize='large')
#ax.add_artist(legend1)  # Add back the first legend

plt.tight_layout()
plt.show()

In [ ]:
sample_of_interest = 'PKD4_humphreys_ADPKD'

#max_value = 300

ct_plot1 = "DCT_CNT_CD"

ct_plot2 = "TAL"

ct_plot3 = "PT"


# Replace with your actual file paths and cell types
cell_types = ["Podo", "TAL", "DCT_CNT_CD", "EC", "Stromal", "Immune", "PEC", "PT", "IC", "DTL_ATL"] # etc.
# Paths as specified in your first script
base_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/human_extended/output_human_extended/"
r2_folder = "R2_GO_unfiltered/"
pval_folder = "Padj_GO_unfiltered/"
metadat_path = "/home/isilon/users/o_kloetzer/Atlas/scSPECTRA/R2_pval/Atlas_Extended_II_Albuminuria_gpt.csv"  # Update with the actual path to 'metadat'

def extract_cell_type(gene_set):
    for cell_type in cell_types:
        if gene_set.startswith(cell_type):
            return cell_type
    return "Unknown"  # or some default value

# Read metadata
metadat = pd.read_csv(metadat_path)
unique_samples = metadat[metadat['Disease_level2'] != "Control"]['Sample'].unique()
#unique_samples = metadat[(metadat['Disease_level2'] != "Control") & (metadat['Project/Dataset'] == "cal_CKD")]['Sample'].unique()

# Initialize combined dataframes for R2 and p-values
combined_r2 = pd.DataFrame(index=unique_samples)
combined_pval = pd.DataFrame(index=unique_samples)
sample_counts_per_cell_type = {}

# Process each cell type
for cell_type in cell_types:
    r2_path = os.path.join(base_path, r2_folder, f"R2_{cell_type}.csv")
    pval_path = os.path.join(base_path, pval_folder, f"Pval_{cell_type}.csv")
    
    r2_df = pd.read_csv(r2_path, index_col=0)
    pval_df = pd.read_csv(pval_path, index_col=0)
    
    # Modify column names
    r2_df.columns = [f"{cell_type}_{col}" for col in r2_df.columns]
    pval_df.columns = [f"{cell_type}_{col}" for col in pval_df.columns]
    
    # Merge data
    combined_r2 = combined_r2.join(r2_df, how='left')
    combined_pval = combined_pval.join(pval_df, how='left')

    # Count samples per cell type
    r2_df = r2_df[r2_df.index.isin(unique_samples)]
    sample_counts_per_cell_type[cell_type] = r2_df.shape[0]

# Handle NA values in p-values
combined_pval.fillna(1, inplace=True)

print(combined_r2)

# Initialize a MinMaxScaler
min_max_scaler = MinMaxScaler()

# Scale the combined_r2 data
# Note: The fit_transform method expects a 2D array, so we use .values to convert the DataFrame
min_max_scaled_r2_values = min_max_scaler.fit_transform(combined_r2.values)

# Create a new DataFrame from the min-max scaled data, preserving the index and column names

combined_r2 = pd.DataFrame(min_max_scaled_r2_values, index=combined_r2.index, columns=combined_r2.columns)

# Set p-value threshold
p_value_threshold = 0.05

# Identify significant gene sets
significant_matrix = combined_pval < p_value_threshold

# Count significant occurrences
significant_counts = significant_matrix.sum()

# Create DataFrame for counts and cell types
significant_counts_df = pd.DataFrame({'GeneSet': significant_counts.index, 'Count': significant_counts.values})

# Apply the function to extract the full cell type name
significant_counts_df['CellType'] = significant_counts_df['GeneSet'].apply(extract_cell_type)

# Normalize by sample number per cell type
significant_counts_df['NormalizedPercentage'] = significant_counts_df.apply(
    lambda row: (row['Count'] / sample_counts_per_cell_type.get(row['CellType'], 1)) * 100, axis=1
)

# Display final DataFrame
print(significant_counts_df.head())  # You can modify this to display as needed

sorted_df = significant_counts_df.sort_values(by='NormalizedPercentage', ascending=False)
display(sorted_df)

import re

def auto_split_label(label, max_length=40):
    # Split the label into words
    words = label.split()

    # Function to remove suffix
    def remove_suffix(word):
        return re.sub(r'\(GO:\d{7}\)$', '', word)

    # Start with the first word (with suffix removed)
    first_word = remove_suffix(words[0])
    split_label = first_word
    current_length = len(first_word)

    # Add each word to the line until the max_length is reached, then start a new line
    for word in words[1:]:
        word = remove_suffix(word)
        if current_length + len(word) + 1 <= max_length:
            split_label += ' ' + word
            current_length += len(word) + 1
        else:
            split_label += '\n' + word
            current_length = len(word)

    return split_label


# Provided cell types and colors
cell_colors = {
    "DCT_CNT_CD": "#3182bd",
    "DTL_ATL": "#fdd0a2",
    "EC": "seagreen",
    "IC": "orange",
    "Immune": "#c7e9c0",
    "Podo": "#000004",
    "Stromal": "limegreen",
    "PEC": "#fde725",
    "PT": "darkorchid",
    "TAL": "lightcoral",
}

# Function to extract cell type from feature name
def extract_cell_type(feature_name):
    for cell_type in cell_colors.keys():
        if feature_name.startswith(cell_type):
            return cell_type
    return "Unknown"

# Sample data - replace with your actual data

top_20_features = combined_pval.loc[sample_of_interest].nsmallest(25)
neg_log_pvals = -np.log10(top_20_features)
r2_values = combined_r2.loc[sample_of_interest][top_20_features.index]

# Create a reversed red colormap for R2 values
cmap = plt.cm.Reds_r  # '_r' suffix to reverse the colormap

norm = mcolors.Normalize(vmin=0, vmax=1)  # Normalization fixed from 0 to 1

# Assuming 'significant_counts_df' is your DataFrame and it has a 'GeneSet' and 'NormalizedPercentage' column
scale_factor = 4
base_size = 100
#base_size = 20

# Map features to their normalized percentage
feature_to_percentage = dict(zip(significant_counts_df['GeneSet'], significant_counts_df['NormalizedPercentage']))

# Create the dot plot
fig, ax = plt.subplots(figsize=(16, 5))
for i, (feature, neg_log_pval) in enumerate(neg_log_pvals.items()):
    cell_type = extract_cell_type(feature)
    bar_color = cell_colors.get(cell_type, 'grey')
    dot_color = cmap(norm(r2_values[feature]))
    
    # Get the normalized percentage for the dot size
    normalized_percentage = feature_to_percentage.get(feature, 0)
    dot_size = normalized_percentage * scale_factor + base_size

    ax.plot([0, neg_log_pval - (neg_log_pval / 10)], [i, i], color=bar_color, linewidth=3)
    ax.scatter(neg_log_pval, i, s=dot_size, color=dot_color, edgecolor='black', alpha=0.7, linewidth=0.5)

ax.set_yticks(range(len(top_20_features.index)))
ax.set_yticklabels(top_20_features.index, fontsize=10)
ax.set_xlabel('-log10(p-value)', fontsize=12)
ax.set_ylabel('Features', fontsize=12)
ax.set_title(f'Top Features for Sample {sample_of_interest}', fontsize=14)

# Invert y-axis to have the most significant features at the top
ax.invert_yaxis()

# Create a colorbar for the R2 values
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax, label='Scaled R2 Values', orientation='vertical')

# Legend for cell types
cell_legend_elements = [plt.Line2D([0], [0], color=color, lw=4, label=cell_type) for cell_type, color in cell_colors.items()]
legend1 = ax.legend(handles=cell_legend_elements, title='Cell Types', bbox_to_anchor=(1.5, 1), loc='upper left')

# Example normalized percentages for the dot size legend
example_percentages = [20, 40, 60, 80]
legend_dot_sizes = [p * scale_factor + base_size for p in example_percentages]

# Add legend for dot sizes
for p, size in zip(example_percentages, legend_dot_sizes):
    ax.scatter([], [], s=size, color='gray', edgecolor='black', alpha=0.7, label=f'{p}%')

legend2 = ax.legend(title='Frequency', bbox_to_anchor=(1.5, 0.2), loc='center left')
ax.add_artist(legend1)  # Add back the first legend

plt.tight_layout()
plt.show()


# Define your desired order of cell types
desired_order = [
     
"PT", 
"TAL", 
"DCT_CNT_CD", 
"IC",    
"EC",
"Stromal",      
"Immune",   
"DTL_ATL",  
"PEC",           
"Podo",    

    
]  # Replace with actual cell types

#top_20_features = combined_pval.loc[sample_of_interest].nsmallest(100)

sample_features = combined_pval.loc[sample_of_interest]

top_20_features = sample_features[sample_features < p_value_threshold]

# Assume 'combined_pval' is a DataFrame you have that contains the p-values

# Select the top 50 gene sets from the top features
top_gene_sets = top_20_features.index

# Group the top gene sets by cell type
grouped_data = pd.Series(top_gene_sets).apply(extract_cell_type).value_counts()

categories = list(grouped_data.index)

# Initialize ordered values with zero for all categories
ordered_values = [grouped_data.get(ct, 0) for ct in desired_order]

# Since we need to repeat the first value to close the circular graph
ordered_values += ordered_values[:1]

# Calculate the angle of each axis in the plot
N = len(desired_order)
angles = [n / float(N) * 2 * np.pi for n in range(N)]
angles += angles[:1]

# Initialize the spider plot
fig, ax = plt.subplots(figsize=(4, 4), subplot_kw=dict(polar=True))

# Draw one axis per variable and add labels
plt.xticks(angles[:-1], desired_order)

# Draw ylabels and set plot limits
ax.set_rlabel_position(0)
plt.yticks(color="grey", size=5)


#plt.ylim(0, max_value)


# Neutral color for shading
shade_color = 'lightgrey'

# Fill the entire area under the radar chart with a neutral color
ax.fill(angles, ordered_values, shade_color, alpha=0.9)

# Draw thin lines connecting the points
ax.plot(angles, ordered_values, color='grey', linewidth=1, linestyle='-', alpha=0.8)

# Plot each line segment in its respective cell type color
for idx in range(N):
    color = cell_colors.get(desired_order[idx], "grey")
    ax.plot([angles[idx], 0], [ordered_values[idx], 0], color=color, linewidth=2)

# Change the color of tick labels to red if the cell type is missing
for idx, label in enumerate(ax.get_xticklabels()):
    if desired_order[idx] not in categories:
        label.set_color('red')

# Add a title
plt.title('Cell Type Distribution in Top Gene Sets for Sample ' + sample_of_interest, size=11, y=1.1)

plt.show()


# Desired cell type for visualization
desired_cell_type = ct_plot1

# Assuming 'combined_pval' and 'combined_r2' are DataFrames with samples as rows and features as columns
# Filter for the specific sample of interest
sample_pvals = combined_pval.loc[sample_of_interest]
sample_r2 = combined_r2.loc[sample_of_interest]

# Filter the top features for the specific sample and cell type
#top_features_for_sample = sample_pvals.nsmallest(25)
filtered_features = sample_pvals[sample_pvals.index.map(extract_cell_type) == desired_cell_type].nsmallest(5)
filtered_neg_log_pvals = -np.log10(filtered_features)
filtered_r2_values = sample_r2[filtered_features.index]

# Now, update the dot plot code to use the filtered data
fig, ax = plt.subplots(figsize=(16, 5))
for i, (feature, neg_log_pval) in enumerate(filtered_neg_log_pvals.items()):
    cell_type = extract_cell_type(feature)
    bar_color = cell_colors.get(cell_type, 'grey')
    dot_color = cmap(norm(filtered_r2_values[feature]))
    
    # Get the normalized percentage for the dot size (if applicable)
    normalized_percentage = feature_to_percentage.get(feature, 0)
    dot_size = normalized_percentage * scale_factor + base_size

    ax.plot([0, neg_log_pval - (neg_log_pval / 10)], [i, i], color=bar_color, linewidth=3)
    ax.scatter(neg_log_pval, i, s=dot_size, color=dot_color, edgecolor='black', alpha=0.7, linewidth=0.5)

ax.set_yticks(range(len(filtered_features.index)))

prefix_to_remove = f"{desired_cell_type}_"
cleaned_labels = [label.replace(prefix_to_remove, '', 1) for label in filtered_features.index]

# Apply the auto-splitting to each label
cleaned_labels = [auto_split_label(label) for label in cleaned_labels]

ax.set_yticklabels(cleaned_labels, fontsize=22)
ax.set_xlabel('-log10(p-value)', fontsize=15)
#ax.set_ylabel('Features', fontsize=12)
#ax.set_title(f'Top Features for Sample {sample_of_interest} - Cell Type: {desired_cell_type}', fontsize=14)
ax.invert_yaxis()

# Create a colorbar for the R2 values
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax, label='Scaled R2 Values', orientation='vertical')

# Legend for cell types
#cell_legend_elements = [plt.Line2D([0], [0], color=color, lw=4, label=cell_type) for cell_type, color in cell_colors.items()]
#legend1 = ax.legend(handles=cell_legend_elements, title='Cell Types', bbox_to_anchor=(1.5, 1), loc='upper left')

# Example normalized percentages for the dot size legend
example_percentages = [20, 40, 60, 80]
legend_dot_sizes = [p * scale_factor + base_size for p in example_percentages]

# Add legend for dot sizes
for p, size in zip(example_percentages, legend_dot_sizes):
    ax.scatter([], [], s=size, color='gray', edgecolor='black', alpha=0.7, label=f'{p}%')

legend2 = ax.legend(title='Frequency', bbox_to_anchor=(1.5, 0.2), loc='center left', fontsize='large')
#ax.add_artist(legend1)  # Add back the first legend

plt.tight_layout()
plt.show()




# Desired cell type for visualization
desired_cell_type = ct_plot2

# Assuming 'combined_pval' and 'combined_r2' are DataFrames with samples as rows and features as columns
# Filter for the specific sample of interest
sample_pvals = combined_pval.loc[sample_of_interest]
sample_r2 = combined_r2.loc[sample_of_interest]

# Filter the top features for the specific sample and cell type
#top_features_for_sample = sample_pvals.nsmallest(25)
filtered_features = sample_pvals[sample_pvals.index.map(extract_cell_type) == desired_cell_type].nsmallest(5)
filtered_neg_log_pvals = -np.log10(filtered_features)
filtered_r2_values = sample_r2[filtered_features.index]

# Now, update the dot plot code to use the filtered data
fig, ax = plt.subplots(figsize=(16, 5))
for i, (feature, neg_log_pval) in enumerate(filtered_neg_log_pvals.items()):
    cell_type = extract_cell_type(feature)
    bar_color = cell_colors.get(cell_type, 'grey')
    dot_color = cmap(norm(filtered_r2_values[feature]))
    
    # Get the normalized percentage for the dot size (if applicable)
    normalized_percentage = feature_to_percentage.get(feature, 0)
    dot_size = normalized_percentage * scale_factor + base_size

    ax.plot([0, neg_log_pval - (neg_log_pval / 10)], [i, i], color=bar_color, linewidth=3)
    ax.scatter(neg_log_pval, i, s=dot_size, color=dot_color, edgecolor='black', alpha=0.7, linewidth=0.5)

ax.set_yticks(range(len(filtered_features.index)))

prefix_to_remove = f"{desired_cell_type}_"
cleaned_labels = [label.replace(prefix_to_remove, '', 1) for label in filtered_features.index]

# Apply the auto-splitting to each label
cleaned_labels = [auto_split_label(label) for label in cleaned_labels]

ax.set_yticklabels(cleaned_labels, fontsize=22)
ax.set_xlabel('-log10(p-value)', fontsize=15)
#ax.set_ylabel('Features', fontsize=12)
#ax.set_title(f'Top Features for Sample {sample_of_interest} - Cell Type: {desired_cell_type}', fontsize=14)
ax.invert_yaxis()

# Create a colorbar for the R2 values
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax, label='Scaled R2 Values', orientation='vertical')

# Legend for cell types
#cell_legend_elements = [plt.Line2D([0], [0], color=color, lw=4, label=cell_type) for cell_type, color in cell_colors.items()]
#legend1 = ax.legend(handles=cell_legend_elements, title='Cell Types', bbox_to_anchor=(1.5, 1), loc='upper left')

# Example normalized percentages for the dot size legend
example_percentages = [20, 40, 60, 80]
legend_dot_sizes = [p * scale_factor + base_size for p in example_percentages]

# Add legend for dot sizes
for p, size in zip(example_percentages, legend_dot_sizes):
    ax.scatter([], [], s=size, color='gray', edgecolor='black', alpha=0.7, label=f'{p}%')

legend2 = ax.legend(title='Frequency', bbox_to_anchor=(1.5, 0.2), loc='center left', fontsize='large')
#ax.add_artist(legend1)  # Add back the first legend

plt.tight_layout()
plt.show()





# Desired cell type for visualization
desired_cell_type = ct_plot3

# Assuming 'combined_pval' and 'combined_r2' are DataFrames with samples as rows and features as columns
# Filter for the specific sample of interest
sample_pvals = combined_pval.loc[sample_of_interest]
sample_r2 = combined_r2.loc[sample_of_interest]

# Filter the top features for the specific sample and cell type
#top_features_for_sample = sample_pvals.nsmallest(25)
filtered_features = sample_pvals[sample_pvals.index.map(extract_cell_type) == desired_cell_type].nsmallest(5)
filtered_neg_log_pvals = -np.log10(filtered_features)
filtered_r2_values = sample_r2[filtered_features.index]

# Now, update the dot plot code to use the filtered data
fig, ax = plt.subplots(figsize=(16, 5))
for i, (feature, neg_log_pval) in enumerate(filtered_neg_log_pvals.items()):
    cell_type = extract_cell_type(feature)
    bar_color = cell_colors.get(cell_type, 'grey')
    dot_color = cmap(norm(filtered_r2_values[feature]))
    
    # Get the normalized percentage for the dot size (if applicable)
    normalized_percentage = feature_to_percentage.get(feature, 0)
    dot_size = normalized_percentage * scale_factor + base_size

    ax.plot([0, neg_log_pval - (neg_log_pval / 10)], [i, i], color=bar_color, linewidth=3)
    ax.scatter(neg_log_pval, i, s=dot_size, color=dot_color, edgecolor='black', alpha=0.7, linewidth=0.5)

ax.set_yticks(range(len(filtered_features.index)))

prefix_to_remove = f"{desired_cell_type}_"
cleaned_labels = [label.replace(prefix_to_remove, '', 1) for label in filtered_features.index]

# Apply the auto-splitting to each label
cleaned_labels = [auto_split_label(label) for label in cleaned_labels]

ax.set_yticklabels(cleaned_labels, fontsize=22)
ax.set_xlabel('-log10(p-value)', fontsize=15)
#ax.set_ylabel('Features', fontsize=12)
#ax.set_title(f'Top Features for Sample {sample_of_interest} - Cell Type: {desired_cell_type}', fontsize=14)
ax.invert_yaxis()

# Create a colorbar for the R2 values
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax, label='Scaled R2 Values', orientation='vertical')

# Legend for cell types
#cell_legend_elements = [plt.Line2D([0], [0], color=color, lw=4, label=cell_type) for cell_type, color in cell_colors.items()]
#legend1 = ax.legend(handles=cell_legend_elements, title='Cell Types', bbox_to_anchor=(1.5, 1), loc='upper left')

# Example normalized percentages for the dot size legend
example_percentages = [20, 40, 60, 80]
legend_dot_sizes = [p * scale_factor + base_size for p in example_percentages]

# Add legend for dot sizes
for p, size in zip(example_percentages, legend_dot_sizes):
    ax.scatter([], [], s=size, color='gray', edgecolor='black', alpha=0.7, label=f'{p}%')

legend2 = ax.legend(title='Frequency', bbox_to_anchor=(1.5, 0.2), loc='center left', fontsize='large')
#ax.add_artist(legend1)  # Add back the first legend

plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import os



nan_threshold = 1000

metadat = metadata

# Define the cell types and the base paths
#cell_types = ["Cancer", "Stromal", "Endothelial"]
#cell_types = ["Podo", "TAL", "DCT_CNT_CD", "EC", "Stromal", "Immune", "PEC", "PT", "IC", "DTL_ATL"]  # etc.
cell_types = ["TAL"]  # etc.


# Function to read and process data for samples based on GFR threshold
def process_condition(metadat, unique_samples):


    
    
    combined_pval = pd.DataFrame(index=unique_samples)
    for cell_type in cell_types:
        pval_path = os.path.join(base_path, pval_folder, f"Pval_{cell_type}.csv")
        pval_df = pd.read_csv(pval_path, index_col=0)
        pval_df = pval_df[pval_df.index.isin(unique_samples)]
        pval_df.columns = [f"{cell_type}_{col}" for col in pval_df.columns]
        combined_pval = combined_pval.join(pval_df, how='left')
    return combined_pval

# Function to count significance
def count_significance(pval_df, threshold=0.05):
    sig_count = (pval_df < threshold).sum(axis=0, skipna=True)
    nonsig_count = (pval_df >= threshold).sum(axis=0, skipna=True)
    return sig_count, nonsig_count

# Read metadata

#metadata[(metadata['tissue: tumor_primary'] == 1.0) & (metadata['platform'] == "10x") & (metadata['condition'] == "LUSC")]['patient'].unique()

# Process each condition based on GFR threshold
#combined_pval_cond1 = process_condition(metadata, metadata[(metadata['platform'] == "10x") & (metadata['condition'] == "LUAD")]["patient"].tolist())  # Samples above the threshold
#combined_pval_cond2 = process_condition(metadata, metadata[(metadata['platform'] == "10x") & (metadata['condition'] == "LUSC")]["patient"].tolist())  # Samples below the threshold

combined_pval_cond1 = process_condition(metadata, metadata[(metadata['Disease_level1'] != "Control") & (metadata['Sex'] == "male")]["Sample"].tolist())  # Samples above the threshold
combined_pval_cond2 = process_condition(metadata, metadata[(metadata['Disease_level1'] != "Control") & (metadata['Sex'] == "female")]["Sample"].tolist())  # Samples below the threshold


#filter out the samples with too many nan
# Count the number of NaN values per sample (row) in the combined_pval DataFrame
nan_counts_per_sample1 = combined_pval_cond1.isna().sum(axis=1)
nan_counts_per_sample2 = combined_pval_cond2.isna().sum(axis=1)

# Exclude samples with more than the defined threshold of NaNs
samples_to_exclude1 = nan_counts_per_sample1[nan_counts_per_sample1 > nan_threshold].index
samples_to_exclude2 = nan_counts_per_sample2[nan_counts_per_sample2 > nan_threshold].index

combined_pval_cond1 = combined_pval_cond1.drop(samples_to_exclude1, axis=0)
combined_pval_cond2 = combined_pval_cond2.drop(samples_to_exclude2, axis=0)


# Count significance for each condition
sig_count_cond1, nonsig_count_cond1 = count_significance(combined_pval_cond1)
sig_count_cond2, nonsig_count_cond2 = count_significance(combined_pval_cond2)

# Creating the comparison DataFrame
comparison_df = pd.DataFrame(columns=combined_pval_cond1.columns, index=['cond1_sig', 'cond1_nonsig', 'cond2_sig', 'cond2_nonsig'])
comparison_df.loc['cond1_sig'] = sig_count_cond1
comparison_df.loc['cond1_nonsig'] = nonsig_count_cond1
comparison_df.loc['cond2_sig'] = sig_count_cond2
comparison_df.loc['cond2_nonsig'] = nonsig_count_cond2


import pandas as pd
import numpy as np
from scipy.stats import fisher_exact

# Assuming the comparison_df has been generated as in the previous code

fisher_results = pd.DataFrame(columns=['Feature', 'Test_statistic', 'p_value', "odds_ratio", 'cond1_sig_count', 'cond2_sig_count'])

for feature in comparison_df.columns:
    # Creating the contingency table for each feature
    contingency_table = comparison_df[[feature]].values.reshape(2, 2)
    
    # Extract counts of significant samples for each condition for the current feature
    cond1_sig_count = comparison_df.loc['cond1_sig', feature]
    cond2_sig_count = comparison_df.loc['cond2_sig', feature]

    # Apply Fisher's Exact Test
    odds_ratio, p = fisher_exact(contingency_table)
    
    # Store the results
    result_row = pd.DataFrame([{
        'Feature': feature, 
        'Test_statistic': odds_ratio, 
        'p_value': p, 
        "odds_ratio": odds_ratio, 
        'cond1_sig_count': cond1_sig_count,
        'cond2_sig_count': cond2_sig_count
    }])
    
    fisher_results = pd.concat([fisher_results, result_row], ignore_index=True)

# Sort the results by p-value in ascending order
sorted_results = fisher_results.sort_values(by='p_value')

# Display the most significant features
print(sorted_results.head())  # Adjust the number inside head() as needed

In [ ]:
from statsmodels.stats.multitest import multipletests
# Extract the original p-values
p_values = sorted_results['p_value'].values

# Apply multiple test correction (using Benjamini-Hochberg method as an example)
correction_method = 'fdr_bh'  # You can change this to 'bonferroni', 'holm', etc., based on your requirement
rejected, corrected_p_values, _, _ = multipletests(p_values, alpha=0.05, method=correction_method)

# Add the corrected p-values to your DataFrame
sorted_results['corrected_p_value'] = corrected_p_values
sorted_results['rejected_h0'] = rejected

# Now sorted_results has an additional column with corrected p-values
print(sorted_results.head())  # Adjust the number inside head() as needed

sorted_results.head(100)

In [ ]:
import seaborn as sns

# Define the path to the R2 file for the specific cell type and gene set
#r2_base_path = "/home/kloetzer/Atlas/scSpectra/species_healthy_disease/"
cell_type = "TAL"  # Replace with your desired cell type
target_gene_set = "Postsynaptic Membrane Organization (GO:0001941)"  # Replace with your desired gene set name

metadata = metadat

# Load the R2 file
r2_path = os.path.join(base_path, r2_folder, f"R2_{cell_type}.csv")
pval_path = os.path.join(base_path, pval_folder, f"Pval_{cell_type}.csv")

r2_df = pd.read_csv(r2_path, index_col=0)
pval_df = pd.read_csv(pval_path, index_col=0)

# Define the sample groups (time points)

#group0 = metadata[(metadata["proj"]=="m_humphreys_DKD") & (metadata["treated"]=="Control_diseased")].orig_ident.unique().tolist()
#group0 = metadata[(metadata['Hypertension']=="Yes")].Sample.unique().tolist()

#group1 = metadata[(metadata["proj"]=="m_humphreys_DKD") & (metadata["treated"]=="SLGT2i")].orig_ident.unique().tolist()
group1 = metadata[(metadata['Sex']=="female") & (metadata['Disease_level1']!="Control")].Sample.unique().tolist()

group2 = metadata[(metadata['Sex']=="male") & (metadata['Disease_level1']!="Control")].Sample.unique().tolist()


samples_to_remove = []
samples_to_remove = ['34-10240', 'HK2886']

# Creating the list with samples to be removed
#group0 = [sample for sample in group0 if sample not in samples_to_remove]

group1 = [sample for sample in group1 if sample not in samples_to_remove]

group2 = [sample for sample in group2 if sample not in samples_to_remove]

# Identify significant samples based on Pval < 0.05 for the specific gene set
significant_samples = pval_df.index[pval_df[target_gene_set] < 0.05]


# Assuming the rest of your setup code (loading data, etc.) is here

# Define x-coordinates for the groups, closer together
group_positions = [0.5, 0.7]  # Adjust these values as needed

# Create a figure for the plot
plt.figure(figsize=(2.5, 6))

# Initialize lists to store mean R2 values for each group
mean_r2_values = []

# Plot each individual sample at the new group positions
for i, group in enumerate([group1, group2]):
    group_r2_values = r2_df.loc[group, target_gene_set]
    mean_r2 = group_r2_values.mean()
    mean_r2_values.append(mean_r2)

    for sample in group:
        x_position = group_positions[i]  # x-coordinate for this group
        color = 'red' if sample in significant_samples else 'blue'
        plt.plot(x_position, group_r2_values.loc[sample], marker='o', markersize=5, color=color)

# Plot the mean R2 values
plt.plot(group_positions, mean_r2_values, marker='_', color='black', linestyle='', label='Mean R2')

# Customize the plot
plt.ylim(0.0, 1)  # Set the y-axis range from 0 to 1
plt.xlim(0.3, 0.9)  # Set the x-axis range
plt.xlabel(cell_type)
plt.ylabel("R2 Value")
plt.title(target_gene_set)
plt.xticks(group_positions, ["female", "male"])  # Set custom x-axis ticks
#plt.legend()
plt.grid(False)
plt.show()



# Create a DataFrame for plotting
plot_data = pd.DataFrame()
for i, group in enumerate([group1, group2]):
    group_r2_values = r2_df.loc[group, target_gene_set]
    group_data = pd.DataFrame({'R2 Value': group_r2_values, 'Group': i, 'Sample': group_r2_values.index})
    plot_data = pd.concat([plot_data, group_data])

# Add a column for color based on significance
plot_data['Color'] = plot_data['Sample'].apply(lambda x: 'red' if x in significant_samples else 'blue')

# Create a figure for the plot
plt.figure(figsize=(2, 6))

# Create a beeswarm plot

sns.swarmplot(x='Group', y='R2 Value', data=plot_data, hue='Color', palette=['blue', 'red'])
plt.plot(mean_r2_values, marker='_', color='black', linestyle='', label='Mean R2', zorder=10, markersize=12,
        markeredgewidth=2)

# Customize the plot
plt.ylim(0.5, 1)  # Set the y-axis range from 0 to 1
plt.xlabel(cell_type)
plt.ylabel("R2 Value")
plt.title(target_gene_set)
plt.xticks([0, 1], ["female", "male"])  # Set custom x-axis ticks
plt.legend().remove()  # Remove the legend if not needed
plt.grid(False)
plt.plot(mean_r2_values, marker='_', color='black', linestyle='', label='Mean R2')

plt.show()

In [ ]:
import pandas as pd
import os



nan_threshold = 1000

metadat = metadata

# Define the cell types and the base paths
#cell_types = ["Cancer", "Stromal", "Endothelial"]
cell_types = ["DCT_CNT_CD"]


# Function to read and process data for samples based on GFR threshold
def process_condition(metadat, unique_samples):


    
    
    combined_pval = pd.DataFrame(index=unique_samples)
    for cell_type in cell_types:
        pval_path = os.path.join(base_path, pval_folder, f"Pval_{cell_type}.csv")
        pval_df = pd.read_csv(pval_path, index_col=0)
        pval_df = pval_df[pval_df.index.isin(unique_samples)]
        pval_df.columns = [f"{cell_type}_{col}" for col in pval_df.columns]
        combined_pval = combined_pval.join(pval_df, how='left')
    return combined_pval

# Function to count significance
def count_significance(pval_df, threshold=0.05):
    sig_count = (pval_df < threshold).sum(axis=0, skipna=True)
    nonsig_count = (pval_df >= threshold).sum(axis=0, skipna=True)
    return sig_count, nonsig_count

# Read metadata

#metadata[(metadata['tissue: tumor_primary'] == 1.0) & (metadata['platform'] == "10x") & (metadata['condition'] == "LUSC")]['patient'].unique()

# Process each condition based on GFR threshold
#combined_pval_cond1 = process_condition(metadata, metadata[(metadata['platform'] == "10x") & (metadata['condition'] == "LUAD")]["patient"].tolist())  # Samples above the threshold
#combined_pval_cond2 = process_condition(metadata, metadata[(metadata['platform'] == "10x") & (metadata['condition'] == "LUSC")]["patient"].tolist())  # Samples below the threshold

combined_pval_cond1 = process_condition(metadata, metadata[(metadata['Disease_level1'] != "Control") & (metadata['Hypertension'] == "No")]["Sample"].tolist())  # Samples above the threshold
combined_pval_cond2 = process_condition(metadata, metadata[(metadata['Disease_level1'] != "Control") & (metadata['Hypertension'] == "Yes")]["Sample"].tolist())  # Samples below the threshold


#filter out the samples with too many nan
# Count the number of NaN values per sample (row) in the combined_pval DataFrame
nan_counts_per_sample1 = combined_pval_cond1.isna().sum(axis=1)
nan_counts_per_sample2 = combined_pval_cond2.isna().sum(axis=1)

# Exclude samples with more than the defined threshold of NaNs
samples_to_exclude1 = nan_counts_per_sample1[nan_counts_per_sample1 > nan_threshold].index
samples_to_exclude2 = nan_counts_per_sample2[nan_counts_per_sample2 > nan_threshold].index

combined_pval_cond1 = combined_pval_cond1.drop(samples_to_exclude1, axis=0)
combined_pval_cond2 = combined_pval_cond2.drop(samples_to_exclude2, axis=0)


# Count significance for each condition
sig_count_cond1, nonsig_count_cond1 = count_significance(combined_pval_cond1)
sig_count_cond2, nonsig_count_cond2 = count_significance(combined_pval_cond2)

# Creating the comparison DataFrame
comparison_df = pd.DataFrame(columns=combined_pval_cond1.columns, index=['cond1_sig', 'cond1_nonsig', 'cond2_sig', 'cond2_nonsig'])
comparison_df.loc['cond1_sig'] = sig_count_cond1
comparison_df.loc['cond1_nonsig'] = nonsig_count_cond1
comparison_df.loc['cond2_sig'] = sig_count_cond2
comparison_df.loc['cond2_nonsig'] = nonsig_count_cond2


import pandas as pd
import numpy as np
from scipy.stats import fisher_exact

# Assuming the comparison_df has been generated as in the previous code

fisher_results = pd.DataFrame(columns=['Feature', 'Test_statistic', 'p_value', "odds_ratio", 'cond1_sig_count', 'cond2_sig_count'])

for feature in comparison_df.columns:
    # Creating the contingency table for each feature
    contingency_table = comparison_df[[feature]].values.reshape(2, 2)
    
    # Extract counts of significant samples for each condition for the current feature
    cond1_sig_count = comparison_df.loc['cond1_sig', feature]
    cond2_sig_count = comparison_df.loc['cond2_sig', feature]

    # Apply Fisher's Exact Test
    odds_ratio, p = fisher_exact(contingency_table)
    
    # Store the results
    result_row = pd.DataFrame([{
        'Feature': feature, 
        'Test_statistic': odds_ratio, 
        'p_value': p, 
        "odds_ratio": odds_ratio, 
        'cond1_sig_count': cond1_sig_count,
        'cond2_sig_count': cond2_sig_count
    }])
    
    fisher_results = pd.concat([fisher_results, result_row], ignore_index=True)

# Sort the results by p-value in ascending order
sorted_results = fisher_results.sort_values(by='p_value')

# Display the most significant features
print(sorted_results.head())  # Adjust the number inside head() as needed

In [ ]:
from statsmodels.stats.multitest import multipletests
# Extract the original p-values
p_values = sorted_results['p_value'].values

# Apply multiple test correction (using Benjamini-Hochberg method as an example)
correction_method = 'fdr_bh'  # You can change this to 'bonferroni', 'holm', etc., based on your requirement
rejected, corrected_p_values, _, _ = multipletests(p_values, alpha=0.05, method=correction_method)

# Add the corrected p-values to your DataFrame
sorted_results['corrected_p_value'] = corrected_p_values
sorted_results['rejected_h0'] = rejected

# Now sorted_results has an additional column with corrected p-values
print(sorted_results.head())  # Adjust the number inside head() as needed

sorted_results.head(100)

In [ ]:
import seaborn as sns

# Define the path to the R2 file for the specific cell type and gene set
#r2_base_path = "/home/kloetzer/Atlas/scSpectra/species_healthy_disease/"
cell_type = "DCT_CNT_CD"  # Replace with your desired cell type
target_gene_set = "Regulation Of Monoatomic Ion Transmembrane Transport (GO:0034765)"  # Replace with your desired gene set name

metadata = metadat

# Load the R2 file
r2_path = os.path.join(base_path, r2_folder, f"R2_{cell_type}.csv")
pval_path = os.path.join(base_path, pval_folder, f"Pval_{cell_type}.csv")

r2_df = pd.read_csv(r2_path, index_col=0)
pval_df = pd.read_csv(pval_path, index_col=0)

# Define the sample groups (time points)

#group0 = metadata[(metadata["proj"]=="m_humphreys_DKD") & (metadata["treated"]=="Control_diseased")].orig_ident.unique().tolist()
#group0 = metadata[(metadata['Hypertension']=="Yes")].Sample.unique().tolist()

#group1 = metadata[(metadata["proj"]=="m_humphreys_DKD") & (metadata["treated"]=="SLGT2i")].orig_ident.unique().tolist()
group1 = metadata[(metadata['Hypertension']=="No") & (metadata['Disease_level1']!="Control")].Sample.unique().tolist()

group2 = metadata[(metadata['Hypertension']=="Yes") & (metadata['Disease_level1']!="Control")].Sample.unique().tolist()


samples_to_remove = []
samples_to_remove = ['HK2886', '32-10346']

# Creating the list with samples to be removed
#group0 = [sample for sample in group0 if sample not in samples_to_remove]

group1 = [sample for sample in group1 if sample not in samples_to_remove]

group2 = [sample for sample in group2 if sample not in samples_to_remove]

# Identify significant samples based on Pval < 0.05 for the specific gene set
significant_samples = pval_df.index[pval_df[target_gene_set] < 0.05]


# Assuming the rest of your setup code (loading data, etc.) is here

# Define x-coordinates for the groups, closer together
group_positions = [0.5, 0.7]  # Adjust these values as needed

# Create a figure for the plot
plt.figure(figsize=(2.5, 6))

# Initialize lists to store mean R2 values for each group
mean_r2_values = []

# Plot each individual sample at the new group positions
for i, group in enumerate([group1, group2]):
    group_r2_values = r2_df.loc[group, target_gene_set]
    mean_r2 = group_r2_values.mean()
    mean_r2_values.append(mean_r2)

    for sample in group:
        x_position = group_positions[i]  # x-coordinate for this group
        color = 'red' if sample in significant_samples else 'blue'
        plt.plot(x_position, group_r2_values.loc[sample], marker='o', markersize=5, color=color)

# Plot the mean R2 values
plt.plot(group_positions, mean_r2_values, marker='_', color='black', linestyle='', label='Mean R2')

# Customize the plot
plt.ylim(0.5, 1)  # Set the y-axis range from 0 to 1
plt.xlim(0.3, 0.9)  # Set the x-axis range
plt.xlabel(cell_type)
plt.ylabel("R2 Value")
plt.title(target_gene_set)
plt.xticks(group_positions, ["No", "Yes"])  # Set custom x-axis ticks
#plt.legend()
plt.grid(False)
plt.show()



# Create a DataFrame for plotting
plot_data = pd.DataFrame()
for i, group in enumerate([group1, group2]):
    group_r2_values = r2_df.loc[group, target_gene_set]
    group_data = pd.DataFrame({'R2 Value': group_r2_values, 'Group': i, 'Sample': group_r2_values.index})
    plot_data = pd.concat([plot_data, group_data])

# Add a column for color based on significance
plot_data['Color'] = plot_data['Sample'].apply(lambda x: 'red' if x in significant_samples else 'blue')

# Create a figure for the plot
plt.figure(figsize=(2, 6))

# Create a beeswarm plot

sns.swarmplot(x='Group', y='R2 Value', data=plot_data, hue='Color', palette=['blue', 'red'])
plt.plot(mean_r2_values, marker='_', color='black', linestyle='', label='Mean R2', zorder=10, markersize=12,
        markeredgewidth=2)

# Customize the plot
plt.ylim(0.5, 1)  # Set the y-axis range from 0 to 1
plt.xlabel(cell_type)
plt.ylabel("R2 Value")
plt.title(target_gene_set)
plt.xticks([0, 1], ["No", "Yes"])  # Set custom x-axis ticks
plt.legend().remove()  # Remove the legend if not needed
plt.grid(False)
plt.plot(mean_r2_values, marker='_', color='black', linestyle='', label='Mean R2')

plt.show()